## SANDAG CV and TNC Survey Analysis
### CV: Commercial Vehicles
### TNC: Transportation Network Companies

#### In the cell below, specify which data set to process:
* 'CV' for Establishment surveys
* 'TNC' for TNC surveys

In [1]:
# Indicate what dataset is to be processed. It should be either 'CV' or 'TNC'.
dataset = 'CV'
# dataset = 'TNC'

# Specify value of time for different vehicle sizes, [Light, Medium, Heavy]. Unit is $ per hour.
vot = [67, 68, 89]

#### Import Libraries 

In [2]:
import os, glob
import pandas as pd
import numpy as np
import openpyxl
import datetime as dt
import collections
import matplotlib.pyplot as plt
import openmatrix as omx
import gc

# from matplotlib.ticker import PercentFormatter
# from scipy import stats  # to get inverse of '.quantile()'

In [3]:
from datetime import datetime
current_date = datetime.now().strftime('%Y%m%d')

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### Define Trip Class

In [5]:
class Trip:
    registry = []

    def __init__(self):
        Trip.registry.append(self)
        self.index = -1
        self.o_act = 0
        self.o_place_type = 0
        self.o_place_name = ''
        self.o_address = ''
        self.o_lon = 0
        self.o_lat = 0
        self.o_taz = -1
        self.d_act = 0
        self.d_place_type = 0
        self.d_place_name = ''
        self.d_address = ''
        self.d_lon = 0
        self.d_lat = 0
        self.d_taz = -1
        self.trip_dist = 0
        self.o_dt = dt.datetime(1982, 1, 1, 0, 0, 0)
        self.d_at = dt.datetime(1982, 1, 1, 0, 0, 0)
        self.nt_dt = dt.datetime(1982, 1, 1, 0, 0, 0)
        self.travel_time = 0    # in seconds
        self.stop_duration = 0  # in seconds
        self.cargo_pickup = -1
        self.cargo_delivery = -1
        self.travel_date = dt.date(1982, 1, 1)
        self.last_trip = None

#### Set File Paths

In [6]:
project_path = os.getcwd().replace("\\02 Scripts", "")

# To run the script on your computer, update 'project path' above with script folder address on your computer and
# then uncomment it.
in_data_dir  = '01 Inputs'
out_data_dir = '03 Outputs'

lu_file = 'Lookups_v8.xlsx'
lu_path = os.path.join(project_path, in_data_dir, lu_file)

skims_dir = r'C:\Users\jgliebe\OneDrive - Cambridge Systematics\Documents - PROJ SANDAG Commercial Vehicle & Heavy Truck Model Update\_Shared_CSTeam\Task03_DataID_Review\ABM3\Skims'

if dataset == 'TNC':
    in_data_file = 'TNC Travel Survey_Data Submittal_1-19-23.xlsx'
else:
#     in_data_file = 'SANDAG 2022 CV DataBase & Dictionaires_03_03_2023.xlsx'    
    in_data_file = 'SANDAG 2022 CV DataBase, Revised.xlsx'  # Cleaned up some data formatting
in_data_path = os.path.join(project_path, in_data_dir, in_data_file)

#### Read the Lookup Tables

In [7]:
lookups = pd.read_excel(lu_path, header=0, sheet_name=None)

In [8]:
lookups['Segment Codes']

,Activities,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Places,Unnamed: 6
0,act_seg_code,act_seg_let,act_seg_name,NaN,NaN,plc_seg_code,plc_seg_name
1,1,S,Service,NaN,NaN,1,Residential
2,2,P,Goods_Pickup,NaN,NaN,2,Office
3,3,D,Goods_Delivery,NaN,NaN,3,Warehouse
4,4,M,Maintenance/Other,NaN,NaN,4,Other
5,5,B,Base,NaN,NaN,5,Retail and Restaurant
6,6,H,Home,NaN,NaN,6,Gas
7,NaN,NaN,NaN,NaN,NaN,7,"Industrial, Agriculture, or Construction"
8,NaN,NaN,NaN,NaN,NaN,8,Truck Terminal or Parking


In [9]:
lookup_at = lookups['Activity Type']  # .copy()
# lookup_at.drop(columns='count', inplace=True)

asl = {1: 'S', 2: 'P', 3: 'D', 4: 'M', 5: 'B', 6: 'H'} # asl = Activity Segment Letter
lookup_at['act_seg_let'] = lookup_at['act_seg_code'].map(asl)

dic_at = {}
for i, row in lookup_at.iterrows():
    dic_at[row['activity_type_code']] = row['act_seg_let'], row['act_seg_name']

lookup_at

,activity_type_code,count,activity_type_name,act_seg_name,act_seg_code,act_seg_let
0,10,3066,"Providing professional services (legal, medica...",Service,1,S
1,1,2766,Returning to Base Location,Base,5,B
2,5,1948,Delivering cargo - (including grocery and rest...,Goods_Delivery,3,D
3,8,1181,Providing Installation / Maintenance / Repair ...,Service,1,S
4,96,941,Other Activity (specify),Maintenance/Other,4,M
5,3,663,"Driver Needs (lunch, restroom, etc)",Maintenance/Other,4,M
6,2,589,"Vehicle Maintenance (fuel/charging, etc)",Maintenance/Other,4,M
7,14,531,Home,Home,6,H
8,6,226,Picking up cargo-(including grocery and restau...,Goods_Pickup,2,P
9,11,191,Shopping for Business (i.e. business/office su...,Maintenance/Other,4,M


In [10]:
lookup_pt = lookups['Place Type']   
# lookup_pt.drop(columns='count', inplace=True)

dic_pt = {}
dic_pt2 = {}
for i, row in lookup_pt.iterrows():
    dic_pt[row['place_type_code']] = row['plc_seg_code'], row['plc_seg_name']
    dic_pt2[row['plc_seg_code']] = row['plc_seg_name']
    
lookup_pt # .head(2)

,place_type_code,count,place_type_name,plc_seg_name,plc_seg_code
0,11,3335,Residential / Home,Residential,1
1,1,2607,Office Building (Non-Government),Office,2
2,14,1179,Warehouse,Warehouse,3
3,96,1175,Other (specify):,Other,4
4,6,972,Retail / Shopping,Retail and Restaurant,5
5,16,578,Gas station,Gas,6
6,5,470,Restaurant,Retail and Restaurant,5
7,7,460,Industrial / Manufacturing,"Industrial, Agriculture, or Construction",7
8,2,317,Government Office Building,Office,2
9,8,293,Medical / Hospital / Dental,Office,2


In [11]:
lookup_ind = lookups['Industries']
lookup_ind #.head(2)

,df_column_name,industry_code,industry_group,industry_code.1
0,base_location_Industry Group,1,Agriculture/Mining,1.0
1,base_location_Industry Group,2,Manufacturing,2.0
2,base_location_Industry Group,3,Industrial/Utilities,3.0
3,base_location_Industry Group,4,Retail,4.0
4,base_location_Industry Group,5,Wholesale,5.0
5,base_location_Industry Group,6,Construction,6.0
6,base_location_Industry Group,7,Transportation,7.0
7,base_location_Industry Group,8,Info/Finance/Insurance/Real Estate/Professiona...,8.0
8,base_location_Industry Group,9,Education/Other public services,9.0
9,base_location_Industry Group,10,Medical/Health Services,10.0


In [12]:
lookup_tnc = lookups['TNC Categories']  # .copy()

dic_tnc_names = {}
dic_tnc_categories = {}
for i, row in lookup_tnc.iterrows():
    dic_tnc_names[row['company_name']] = row['company_name_alt']   
    dic_tnc_categories[row['company_name_alt']] = row['TNC_IndCat3']   

lookup_tnc

,company_name,company_name_alt,Estb (ie veh),Trips,Trips/Veh,Tnc_Cat,TNC_IndCat3
0,amazon,Amazon,34.0,748.0,22.000000,Package/Other,NonRestRetl
1,amazonflex,Amazon Flex,15.0,259.0,17.266667,Package/Other,NonRestRetl
2,amazonfresh,Amazon Fresh,1.0,12.0,12.000000,Grocery,Retail
3,axhire,Axle Hire,4.0,61.0,15.250000,Package/Other,NonRestRetl
4,axle,Axle Hire,NaN,NaN,15.250000,Package/Other,NonRestRetl
5,axlehire,Axle Hire,NaN,NaN,15.250000,Package/Other,NonRestRetl
6,doordash,Door Dash,90.0,907.0,10.077778,Restaurant,Restaurant
7,fantuan,Fantuan,1.0,23.0,23.000000,Restaurant,Restaurant
8,gopuff,Go Puff,2.0,38.0,19.000000,Food,Retail
9,grubhub,Grub Hub,43.0,576.0,13.395349,Restaurant,Restaurant


In [13]:
lookup_cv_estab = lookups['CV Estab TAZ']
lookup_cv_estab.head()

,OBJECTID,Join_Count,TARGET_FID,company_id,company_name,company_location_address,company_location_city,company_location_state,company_location_zipcode,company_location_latitude,company_location_longitude,estab_taz
0,1,1,1,100002,ALTAR,4370 LA JOLLA VILLAGE DR # 655,SAN DIEGO,CA,92122,32.873754,-117.210198,1244
1,2,1,2,100003,John Baker Property Mgmt,405 W 9TH AVE,ESCONDIDO,CA,92025,33.112208,-117.080743,3159
2,3,1,3,100004,OWB RANCHES LLC,512 VIA DE LA VALLE # 310,SOLANA BEACH,CA,92075,32.980370,-117.260820,536
3,4,1,4,100005,"United Sportfishers of San Diego, Inc.",2803 Emerson St,San Diego,CA,92106,32.723522,-117.227602,1073
4,5,1,5,100007,SCRIPPS COASTAL RESERVE,9500 GILMAN DR,LA JOLLA,CA,92093,32.877189,-117.237422,952


In [14]:
lookup_tnc_estab = lookups['TNC Estab TAZ']  # .copy()
lookup_tnc_estab.loc[lookup_tnc_estab['estab_taz'].isnull(), 'estab_taz'] = -1 # Could've been done with fillna too.
lookup_tnc_estab['estab_taz'] = lookup_tnc_estab['estab_taz'].astype(int)
lookup_tnc_estab.head()

,OBJECTID,Join_Count,TARGET_FID,company_id,company_name,company_location_address,company_location_city,company_location_state,company_location_zipcode,company_location_latitude,company_location_longitude,estab_taz
0,1,1,1,600026,Uber Eats,1052 Woodlawn Ave,Chula Vista,California,91911,32.610634,-117.086326,2958
1,2,1,2,600027,Amazon Flex,3980 Hatton St,San Diego,California,92111,32.816592,-117.165559,1787
2,3,1,3,600028,Senpex,6881 Alvarado Rd,San Diego,California,92120,32.775523,-117.050791,3502
3,4,1,4,600029,Grub Hub,4437 39th St,San Diego,California,92116,32.757872,-117.110067,2538
4,5,1,5,600031,Uber Eats,5741 Carnegie St,San Diego,California,92122,32.846406,-117.217430,1207


In [15]:
lookup_cv_estab_replace = lookups['CV Rt Rplcmnt Estab TAZ'] #.copy()
lookup_cv_estab_replace.insert(1, 'Date', 0)
lookup_cv_estab_replace.insert(2, 'Veh', 0)
lookup_cv_estab_replace['Date'] = pd.to_datetime(lookup_cv_estab_replace['route_id'].astype(str).str[:8]) #.astype(int)
lookup_cv_estab_replace['Veh'] = lookup_cv_estab_replace['route_id'].astype(str).str[-4:].astype(int)

lookup_cv_estab_replace

,route_id,Date,Veh,NewEstab_TAZ,Revision,Unnamed: 3,Unnamed: 4
0,202206280030,2022-06-28,30,3262,OK,NaN,Note: New Estab TAZes have been assigned to ro...
1,202206280035,2022-06-28,35,3163,OK,NaN,"This means, according to this sheet, two vehic..."
2,202206290038,2022-06-29,38,1184,OK,NaN,Example: Routes 202206280030 and 202206280035 ...
3,202207190119,2022-07-19,119,3607,OK,NaN,NaN
4,202207200224,2022-07-20,224,124,OK,NaN,NaN
5,202207200242,2022-07-20,242,1180,OK,NaN,NaN
6,202207270317,2022-07-27,317,73,OK,NaN,NaN
7,202208020513,2022-08-02,513,1248,OK,NaN,NaN
8,202208030497,2022-08-03,497,3431,OK,NaN,NaN
9,202208030501,2022-08-03,501,3431,OK,NaN,NaN


In [16]:
# Get LogisticsNodes
lookup_logistics = lookups['LogisticsNodes']
lookup_logistics

,FreightCode,Category,TAZ,Description,Label
0,1,Sea Port,2086,Tenth Ave Marine Terminal and BNSF Freight Yard,Seaport
1,1,Sea Port,1154,Tenth Ave Marine Terminal and BNSF Freight Yard,Seaport
2,1,Sea Port,2497,National City Marine Terminal,Seaport
3,1,Sea Port,2193,Ship Building,Seaport
4,2,Airport Cargo,1294,Airport Infrastructure,Airport
5,2,Airport Cargo,1338,Airport Infrastructure,Airport
6,2,Airport Cargo,1457,Airport Infrastructure,Airport
7,2,Airport Cargo,1476,"UPS, FedEx, DHL",Airport
8,2,Airport Cargo,1485,Air Cargo,Airport
9,2,Airport Cargo,1520,Airport Infrastructure,Airport


### Read Skim Matrices -- two methods, choose one
1. Read OMX files, process, and save as pickle files -- do this only if the pickle files do not already exist or new raw data skims are wanted.
2. Read the pickle files directly, assuming they exist. This is 100 times faster.

In [17]:
# Note: There are 4,947 TAZs and 24,321 MGRAs in the shapefiles.
# The OMX files contain info about TAZs, not MGRAs.
dic_veh_size = {1: 'L', 2: 'M', 3: 'H'}
dic_tod = {1: 'EA', 2: 'AM', 3: 'MD', 4: 'PM', 5: 'EV'}
fns = [f'traffic_skims_{v}.omx' for v in dic_tod.values()] # fns = file names
# print(fns)

# Periods:
# Early:   3AM     6AM
# AM Peak: 6AM     9AM
# Midday:  9AM     3:30PM
# PM Peak: 3:30PM  7PM
# Late:    7PM     3AM

dist_dfs = [[None for j in range(len(dic_veh_size))] for i in range(len(dic_tod))]
# 'dist_dfs' is a list of lists that stores distance dataframes. 
# Each row is for a time of day, and each column is a vehicle size.
# Note: Indices start from zero.

time_dfs = [[None for j in range(len(dic_veh_size))] for i in range(len(dic_tod))]
toll_dfs = [[None for j in range(len(dic_veh_size))] for i in range(len(dic_tod))]
g_tt_dfs = [[None for j in range(len(dic_veh_size))] for i in range(len(dic_tod))]

#### Method 1: Read skims for OMX files, process, and save as pickle files. (Slow)
Change below cell to "code" before running

#### Method 2: Read already processed skims from pickle files. (Fast)
Change below cell to "code" before running

In [18]:
# Read skim matrices from pickle files (fast load)
for i in range(len(dic_tod)):
    for j in range(len(dic_veh_size)):
        in_file = f'{3*i+j+1:02}_{dic_tod[i+1]}_{dic_veh_size[j+1]}_10th_of_miles.pkl'
        in_path = os.path.join(project_path, out_data_dir, 'Distance Skims')
        if not os.path.exists(in_path):
           print(f"Input skims file path not found: \n{in_path}")
        in_path = os.path.join(project_path, out_data_dir, 'Distance Skims', in_file)
        dist_dfs[i][j] = pd.read_pickle(in_path)
        
        if i == 2:
            in_file = f'{3*i+j+1:02}_{dic_tod[i+1]}_{dic_veh_size[j+1]}_TT_Minutes.pkl'
            in_path = os.path.join(project_path, out_data_dir, 'Travel Time Skims')
            if not os.path.exists(in_path):
               print(f"Input skims file path not found: \n{in_path}")
            in_path = os.path.join(project_path, out_data_dir, 'Travel Time Skims', in_file)
            time_dfs[i][j] = pd.read_pickle(in_path)
            
            in_file = f'{3*i+j+1:02}_{dic_tod[i+1]}_{dic_veh_size[j+1]}_Toll_Cents.pkl'
            in_path = os.path.join(project_path, out_data_dir, 'Toll Skims')
            if not os.path.exists(in_path):
               print(f"Input skims file path not found: \n{in_path}")
            in_path = os.path.join(project_path, out_data_dir, 'Toll Skims', in_file)
            toll_dfs[i][j] = pd.read_pickle(in_path)
            
            in_file = f'{3*i+j+1:02}_{dic_tod[i+1]}_{dic_veh_size[j+1]}_GenTT_Minutes.pkl'
            in_path = os.path.join(project_path, out_data_dir, 'Generalized TT Skims')
            if not os.path.exists(in_path):
               print(f"Input skims file path not found: \n{in_path}")
            in_path = os.path.join(project_path, out_data_dir, 'Generalized TT Skims', in_file)
            g_tt_dfs[i][j] = pd.read_pickle(in_path)

In [19]:
print(f'Example 1: Distance from TAZ 5 to TAZ 19 in AM peak for medium trucks is: {dist_dfs[1][1].at[320, 344]/10:} miles\n')
print(f'Example 2: Distance from TAZ 321 to TAZ 345 in mid-day for light trucks is: {dist_dfs[2][0].iloc[320, 344]/10:} miles,',
      '\n', f'          Travel time is: {time_dfs[2][0].iloc[320, 344]} minutes, and', '\n',
      f'          toll is: {toll_dfs[2][0].iloc[320, 344]} cents.\n')
print(f'Example 3: Fastest route between TAZ 3901 and TAZ 4233 uses expressway 125, which is a FasTrak toll road.',
      f'\n           Distance from TAZ 3901 to TAZ 4233 in mid-day for heavy trucks is: {dist_dfs[2][2].iloc[3900, 4232]/10:} miles,',
      '\n', f'          Travel time is: {time_dfs[2][2].iloc[3900, 4232]} minutes,\n',
      f'          Toll is: {toll_dfs[2][2].iloc[3900, 4232]} cents, and\n'
      f'           Generalized travel time is: {g_tt_dfs[2][2].iloc[3900, 4232]} minutes.')

Example 1: Distance from TAZ 5 to TAZ 19 in AM peak for medium trucks is: 0.8 miles

Example 2: Distance from TAZ 321 to TAZ 345 in mid-day for light trucks is: 0.9 miles, 
           Travel time is: 2.8 minutes, and 
           toll is: 0 cents.

Example 3: Fastest route between TAZ 3901 and TAZ 4233 uses expressway 125, which is a FasTrak toll road. 
           Distance from TAZ 3901 to TAZ 4233 in mid-day for heavy trucks is: 7.9 miles, 
           Travel time is: 13.63 minutes,
           Toll is: 375 cents, and
           Generalized travel time is: 16.15 minutes.


#### Read Stop, Establishment, and Vehicle Data

In [20]:
# Read the survey data.
df_original = pd.read_excel(in_data_path, header=0, sheet_name=None)
# df_original is a dictionary of Dfs. Keys are sheetnames, and values are the dataframes in those worksheets.

In [21]:
# # If all we were to do was to read the stop data:
# df_stops = pd.read_excel(in_data_path, header=0, sheet_name='Trip Data')
# df_stops.head(2)

In [22]:
df_stops = df_original['Trip Data'].copy()
df_estab = df_original['Establishment Data'].copy()
df_veh = df_original['Vehicle Data'].copy()
df_stops.head()

,unique_id,company_id,vehicle_id,driver_id,trip_number,trip_load_status,trip_load_weight,activity_type,activity_type_other,placetype,placetype_other,location_placename,location_address,location_city,location_state,location_zip,location_latitude,location_longitude,taz,cargo_pickup,cargo_other_pickup,cargo_delivery,cargo_other_delivery,cargo_pu_weight,cargo_do_weight,travel_date,arrival_time,departure_time,used_other_vehicle,participation_type,vehicle_id.1,Lower Estimate Weight Factor,Most Likely Estimate Weight Factor,Upper Estimate Weight Factor
0,1,100012,2062,2589,0,4.0,NaN,1,NaN,11,NaN,BASE,2801 B ST # 55,SAN DIEGO,California,92102,32.717871,-117.133702,2284,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-30,NaN,10:00:00,2,online,2062,109.693404,177.825038,198.177052
1,2,100012,2062,2589,1,NaN,NaN,11,NaN,6,NaN,PET SMART,1660 Camino De La Reina d,San Diego,California,92108,32.770912,-117.149017,2091,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-30,10:20:00,11:10:00,2,online,2062,109.693404,177.825038,198.177052
2,3,100012,2062,2589,2,NaN,NaN,1,NaN,11,NaN,BASE,2801 B ST # 55,SAN DIEGO,California,92102,32.717871,-117.133702,2284,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-30,11:22:00,11:40:00,2,online,2062,109.693404,177.825038,198.177052
3,4,100012,2062,2589,3,NaN,NaN,10,NaN,96,DOG PARK,DOG PARK,2500 Balboa Dr,San Diego,California,92103,32.731546,-117.158124,1933,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-30,12:00:00,14:00:00,2,online,2062,109.693404,177.825038,198.177052
4,5,100012,2062,2589,4,NaN,NaN,1,NaN,11,NaN,BASE,2801 B ST # 55,SAN DIEGO,California,92102,32.717871,-117.133702,2284,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-30,14:20:00,NaN,2,online,2062,109.693404,177.825038,198.177052


In [23]:
df_estab.head()

,company_id,"Industry Group_Size Code (Group from Column D; Size 1=0-9 emp, 2=10+)",company_name,base_location_Industry Group,company_location_address,company_location_city,company_location_state,company_location_zipcode,company_location_latitude,company_location_longitude,employees_fulltime_count,employees_parttime_count,no_of_emp_work,"Number of Employess (1=<10, 2=10+)",total_telecommute_from_home,is_use_tnc,no_of_deliveries,no_of_deliveries_pcsuvpu,no_of_deliveries_su,no_of_deliveries_cu,per_of_deliveries_warehouse,per_of_deliveries_airport,per_of_deliveries_manufacture,per_of_deliveries_retail,per_of_deliveries_service,per_of_deliveries_other,per_of_deliveries_dontknow,per_of_deliveries_text_other,no_of_from_deliveries,no_of_from_deliveries_pcsuvpu,no_of_from_deliveries_su,no_of_from_deliveries_cu,per_of_from_deliveries_warehouse,per_of_from_deliveries_airport,per_of_from_deliveries_manufacture,per_of_from_deliveries_retail,per_of_from_deliveries_service,per_of_from_deliveries_other,per_of_from_deliveries_dontknow,per_of_from_deliveries_text_other,has_vehicles,vehicle_count_total,vehicle_axle_cargotruck,vehicle_semi_cargo,vehicle_cars_count,vehicle_pickuptrucks_count,vehicle_cargovans_count,vehicle_servicevans_count,use_personal_vehicle,vehicle_other_count_total,vehicle_other_axle_cargotruck,vehicle_other_semi_cargo,vehicle_other_cars_count,vehicle_other_pickuptrucks_count,vehicle_other_cargovans_count,vehicle_other_servicevans_count,vehicle_toll_trans_count,Participated in Travel Survey?,LCV Owned or Leased,LCV other,LCV Total,SUT Owned or Leased,SUT other,SUT Total,MUT Owned or Leased,MUT other,MUT Total,TOTAL CVs
0,100002,1_2,ALTAR,1,4370 LA JOLLA VILLAGE DR # 655,SAN DIEGO,CA,92122,32.873754,-117.210198,50,0,50,2,0,2,3,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,NaN,3,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NO,0,0,0,0,0,0,0,0,0,0
1,100003,1_1,John Baker Property Mgmt,1,405 W 9TH AVE,ESCONDIDO,CA,92025,33.112208,-117.080743,2,0,2,1,0,2,2,1.0,1.0,0.0,50.0,0.0,0.0,0.0,0.0,50.0,0.0,NaN,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,NO,0,1,1,0,0,0,0,0,0,1
2,100004,1_1,OWB RANCHES LLC,1,512 VIA DE LA VALLE # 310,SOLANA BEACH,CA,92075,32.980370,-117.260820,3,1,4,1,0,2,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,NaN,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NO,0,0,0,0,0,0,0,0,0,0
3,100005,1_2,"United Sportfishers of San Diego, Inc.",1,2803 Emerson St,San Diego,CA,92106,32.723522,-117.227602,20,14,34,2,2,2,10,9.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,NaN,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,NO,0,0,0,0,1,1,1,0,1,2
4,100007,1_1,SCRIPPS COASTAL RESERVE,1,9500 GILMAN DR,LA JOLLA,CA,92093,32.877189,-117.237422,5,0,5,1,5,2,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,NO,0,1,1,0,0,0,0,0,0,1


In [24]:
df_veh.head()

,id,company_id,veh_make_other,vehicle_model,vehicle_year,vehicle_type,vehicle_fuel_type,vehicle_miles_per_gallon,has_vehicle_transponder,vehicle_classification,vehicle_weight,odometer,ending_odometer,Industry Group,"Employee Size (1=0-9, 2=10+)",Vehicle Type,Vehicle Type_Industry_Size Category Code,Lower Estimate Weight Factor,Most Likely Estimate Weight Factor,Upper Estimate Weight Factor,id.1
0,20,120941,Ford,TRANSIT,2020,2,1,15.0,2.0,3,9999998,9999998,9999998.0,6,1,LCV,LCV_6_1,89.897285,116.750222,120.350116,20
1,21,120941,Ford,TRANSIT,2020,2,1,15.0,2.0,3,9999998,9999998,9999998.0,6,1,LCV,LCV_6_1,89.897285,116.750222,120.350116,21
2,22,120941,Ford,TRANSIT,2020,2,1,15.0,2.0,3,9999998,9999998,9999998.0,6,1,LCV,LCV_6_1,89.897285,116.750222,120.350116,22
3,23,120941,Ford,TRANSIT,2020,3,1,15.0,2.0,3,9999998,9999998,9999998.0,6,1,LCV,LCV_6_1,89.897285,116.750222,120.350116,23
4,24,120941,Ford,TRANSIT,2020,2,1,15.0,2.0,3,9999998,9999998,9999998.0,6,1,LCV,LCV_6_1,89.897285,116.750222,120.350116,24


#### Explore the Vehicle Dataframe

In [25]:
# 'vehicle_classification' codes according to data dictionary:
# 1: Passenger Car or Motorcycle
# 2: Pick-up Truck (4 wheels)
# 3: Van (Cargo/Minivan) (4 wheels)
# 4: Buses 
# 5: Single Unit 2-axle
# 6: Single Unit 3-axle
# 7: Single Unit 4-axle
# 8: Semi (all Tractor-Trailer combinations)
# 96:Other (please specify)

In [26]:
df_veh['vehicle_classification'].unique()

array([3, 2, 6, 1, 5, 8, 4, 7], dtype=int64)

In [27]:
if dataset == 'CV':
    print(df_veh['Vehicle Type'].unique())

['LCV' 'SUT' 'MUT']


In [28]:
if dataset == 'CV':
    pvt_axle_vtype = pd.pivot_table(df_veh, values='id', index='vehicle_classification', columns='Vehicle Type',
                             aggfunc='count') #, sort=True) For sort, default is True.
    print(pvt_axle_vtype)

Vehicle Type              LCV    MUT    SUT
vehicle_classification                     
1                       626.0    NaN    NaN
2                       395.0    NaN    NaN
3                       282.0    NaN    NaN
4                         NaN    NaN   32.0
5                         NaN    NaN  166.0
6                         NaN    NaN   25.0
7                         NaN    NaN    5.0
8                         NaN  220.0    NaN


In [29]:
# Create a conversion dictionary from veh_classification to veh_size.
# This is necessary because the TNC dataset doesn't have the 'Vehicle Type' column - this script
# needs to work for both CV and the TNC datasets.
d1 = {i: 'LCV' for i in range(1, 4)}
d2 = {i: 'SUT' for i in range(4, 8)}
d3 = {8: 'MUT'}
dic_veh_size = {**d1, **d2, **d3}
dic_veh_size

{1: 'LCV',
 2: 'LCV',
 3: 'LCV',
 4: 'SUT',
 5: 'SUT',
 6: 'SUT',
 7: 'SUT',
 8: 'MUT'}

In [30]:
df_veh['veh_size'] = df_veh['vehicle_classification'].map(dic_veh_size)
df_veh[['veh_size', 'vehicle_classification']]

,veh_size,vehicle_classification
0,LCV,3
1,LCV,3
2,LCV,3
3,LCV,3
4,LCV,3
5,LCV,3
6,LCV,3
7,LCV,2
8,LCV,2
9,LCV,2


#### Clean the Data

In [31]:
# Decide which columns to keep and rearrange.
cols = ['company_id', 'vehicle_id', 'driver_id', 'trip_number', 'activity_type', 'placetype',
        'location_placename', 'location_address', 'location_longitude', 'location_latitude', 'taz',
        'arrival_time', 'departure_time', 'cargo_pickup', 'cargo_delivery', 'travel_date', 
        'participation_type', 'Most Likely Estimate Weight Factor']
        # 'Lower Estimate Weight Factor', 
        # 'Upper Estimate Weight Factor'

df_stops = df_stops[cols]

# Sort the dataframe
df_stops.sort_values(by=['company_id', 'vehicle_id', 'travel_date', 'trip_number'],
                     inplace=True, ascending=True)
df_stops.head()

,company_id,vehicle_id,driver_id,trip_number,activity_type,placetype,location_placename,location_address,location_longitude,location_latitude,taz,arrival_time,departure_time,cargo_pickup,cargo_delivery,travel_date,participation_type,Most Likely Estimate Weight Factor
0,100012,2062,2589,0,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,NaN,10:00:00,NaN,NaN,2022-09-30,online,177.825038
1,100012,2062,2589,1,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,10:20:00,11:10:00,NaN,NaN,2022-09-30,online,177.825038
2,100012,2062,2589,2,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11:22:00,11:40:00,NaN,NaN,2022-09-30,online,177.825038
3,100012,2062,2589,3,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,12:00:00,14:00:00,NaN,NaN,2022-09-30,online,177.825038
4,100012,2062,2589,4,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,14:20:00,NaN,NaN,NaN,2022-09-30,online,177.825038


#### Attach Vehicle Type Info

In [32]:
df_stops = df_stops.merge(df_veh[['id', 'veh_size']], how='left', left_on='vehicle_id', right_on='id')
df_stops.rename(columns={'veh_size': 'veh_type'}, inplace=True)
df_stops.drop(columns='id', inplace=True)
df_stops.head()

,company_id,vehicle_id,driver_id,trip_number,activity_type,placetype,location_placename,location_address,location_longitude,location_latitude,taz,arrival_time,departure_time,cargo_pickup,cargo_delivery,travel_date,participation_type,Most Likely Estimate Weight Factor,veh_type
0,100012,2062,2589,0,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,NaN,10:00:00,NaN,NaN,2022-09-30,online,177.825038,LCV
1,100012,2062,2589,1,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,10:20:00,11:10:00,NaN,NaN,2022-09-30,online,177.825038,LCV
2,100012,2062,2589,2,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11:22:00,11:40:00,NaN,NaN,2022-09-30,online,177.825038,LCV
3,100012,2062,2589,3,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,12:00:00,14:00:00,NaN,NaN,2022-09-30,online,177.825038,LCV
4,100012,2062,2589,4,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,14:20:00,NaN,NaN,NaN,2022-09-30,online,177.825038,LCV


#### Attach Establishment TAZ Info

In [33]:
lookup_estab = lookup_cv_estab if dataset == 'CV' else lookup_tnc_estab
df_stops = df_stops.merge(lookup_estab[['company_id', 'estab_taz']], how='left', on='company_id')
df_stops.head()

,company_id,vehicle_id,driver_id,trip_number,activity_type,placetype,location_placename,location_address,location_longitude,location_latitude,taz,arrival_time,departure_time,cargo_pickup,cargo_delivery,travel_date,participation_type,Most Likely Estimate Weight Factor,veh_type,estab_taz
0,100012,2062,2589,0,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,NaN,10:00:00,NaN,NaN,2022-09-30,online,177.825038,LCV,2284
1,100012,2062,2589,1,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,10:20:00,11:10:00,NaN,NaN,2022-09-30,online,177.825038,LCV,2284
2,100012,2062,2589,2,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11:22:00,11:40:00,NaN,NaN,2022-09-30,online,177.825038,LCV,2284
3,100012,2062,2589,3,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,12:00:00,14:00:00,NaN,NaN,2022-09-30,online,177.825038,LCV,2284
4,100012,2062,2589,4,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,14:20:00,NaN,NaN,NaN,2022-09-30,online,177.825038,LCV,2284


#### Update establishment TAZ info for stops that have a replacement establishment TAZ

In [34]:
if dataset == 'CV':
    for i, rowStop in df_stops.iterrows():
        for j, rowRep in lookup_cv_estab_replace.iterrows():
            if rowStop['travel_date']==rowRep['Date'] and rowStop['vehicle_id']==rowRep['Veh']:
#                 print(i, j,
#                       f"{df_stops.at[i, 'estab_taz']} was replaced by {lookup_cv_estab_replace.at[j, 'NewEstab_TAZ']}")
                df_stops.at[i, 'estab_taz'] = lookup_cv_estab_replace.at[j, 'NewEstab_TAZ']                
                break     

#### Create the Trip Dataframe Using the Stop Dataframe

In [35]:
df_temp = df_stops.copy()
rename_dic = {
    'activity_type':      'd_act',
    'placetype':          'd_place_type',
    'location_placename': 'd_place_name',
    'location_address':   'd_address',
    'location_longitude': 'd_lon',
    'location_latitude':  'd_lat',
    'taz':                'd_taz',
    'arrival_time':       'd_arr_time',
    'departure_time':     'next_trip_dep_time',
    'Most Likely Estimate Weight Factor': 'expnsn_factor'
}
df_temp.rename(columns=rename_dic, inplace=True)
df_temp.head()

,company_id,vehicle_id,driver_id,trip_number,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,d_arr_time,next_trip_dep_time,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,veh_type,estab_taz
0,100012,2062,2589,0,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,NaN,10:00:00,NaN,NaN,2022-09-30,online,177.825038,LCV,2284
1,100012,2062,2589,1,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,10:20:00,11:10:00,NaN,NaN,2022-09-30,online,177.825038,LCV,2284
2,100012,2062,2589,2,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11:22:00,11:40:00,NaN,NaN,2022-09-30,online,177.825038,LCV,2284
3,100012,2062,2589,3,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,12:00:00,14:00:00,NaN,NaN,2022-09-30,online,177.825038,LCV,2284
4,100012,2062,2589,4,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,14:20:00,NaN,NaN,NaN,2022-09-30,online,177.825038,LCV,2284


In [36]:
# For TNCs, set missing expansion factors to 1
df_temp['expnsn_factor'] = df_temp['expnsn_factor'].fillna(1)

In [37]:
# Create a dictionary of new columns that should be added.
cols_to_add_1 = ['o_act', 'o_place_type', 'o_lon', 'o_lat', 'o_taz', 'o_dep_time',
                 'travel_time', 'stop_duration',
                 'o_act_seg', 'o_plc_seg', 
                 'd_act_seg', 'd_plc_seg',
                 'trip_dist', 'orgn_to_hq_dist',
                 'hq_taz'
                ]
d1 = dict.fromkeys(cols_to_add_1, 0)

cols_to_add_2 = ['o_place_name', 'o_address',
                 'o_act_seg_name', 'o_plc_seg_name',
                 'd_act_seg_name', 'd_plc_seg_name',
                 'headquarters'
                ]
d2 = dict.fromkeys(cols_to_add_2, "")

d = {**d1, **d2}
d

{'o_act': 0,
 'o_place_type': 0,
 'o_lon': 0,
 'o_lat': 0,
 'o_taz': 0,
 'o_dep_time': 0,
 'travel_time': 0,
 'stop_duration': 0,
 'o_act_seg': 0,
 'o_plc_seg': 0,
 'd_act_seg': 0,
 'd_plc_seg': 0,
 'trip_dist': 0,
 'orgn_to_hq_dist': 0,
 'hq_taz': 0,
 'o_place_name': '',
 'o_address': '',
 'o_act_seg_name': '',
 'o_plc_seg_name': '',
 'd_act_seg_name': '',
 'd_plc_seg_name': '',
 'headquarters': ''}

In [38]:
# Add the new columns with them being initialized by the values of the above dictionary.
df_temp = df_temp.assign(**d)
df_temp.head()

,company_id,vehicle_id,driver_id,trip_number,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,d_arr_time,next_trip_dep_time,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,veh_type,estab_taz,o_act,o_place_type,o_lon,o_lat,o_taz,o_dep_time,travel_time,stop_duration,o_act_seg,o_plc_seg,d_act_seg,d_plc_seg,trip_dist,orgn_to_hq_dist,hq_taz,o_place_name,o_address,o_act_seg_name,o_plc_seg_name,d_act_seg_name,d_plc_seg_name,headquarters
0,100012,2062,2589,0,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,NaN,10:00:00,NaN,NaN,2022-09-30,online,177.825038,LCV,2284,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,,,,,
1,100012,2062,2589,1,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,10:20:00,11:10:00,NaN,NaN,2022-09-30,online,177.825038,LCV,2284,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,,,,,
2,100012,2062,2589,2,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11:22:00,11:40:00,NaN,NaN,2022-09-30,online,177.825038,LCV,2284,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,,,,,
3,100012,2062,2589,3,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,12:00:00,14:00:00,NaN,NaN,2022-09-30,online,177.825038,LCV,2284,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,,,,,
4,100012,2062,2589,4,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,14:20:00,NaN,NaN,NaN,2022-09-30,online,177.825038,LCV,2284,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,,,,,


In [39]:
# Rearrange.
cols = ['company_id', 'vehicle_id', 'driver_id', 'trip_number',
        'o_act_seg', 'o_act_seg_name', 'o_plc_seg', 'o_plc_seg_name',
        'd_act_seg', 'd_act_seg_name', 'd_plc_seg', 'd_plc_seg_name', 
        'o_act', 'o_place_type', 'o_place_name', 'o_address', 'o_lon', 'o_lat', 'o_taz',
        'd_act', 'd_place_type', 'd_place_name', 'd_address', 'd_lon', 'd_lat', 'd_taz',
        'trip_dist', 'headquarters', 'hq_taz', 'orgn_to_hq_dist',
        'o_dep_time', 'd_arr_time', 'next_trip_dep_time', 'travel_time', 'stop_duration',        
        'veh_type', 'cargo_pickup', 'cargo_delivery', 'travel_date',
        'participation_type', 'expnsn_factor', 'estab_taz'
       ]
df_temp = df_temp[cols]
df_temp.head()

,company_id,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,o_dep_time,d_arr_time,next_trip_dep_time,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz
0,100012,2062,2589,0,0,,0,,0,,0,,0,0,,,0,0,0,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,0,NaN,10:00:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
1,100012,2062,2589,1,0,,0,,0,,0,,0,0,,,0,0,0,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,0,,0,0,0,10:20:00,11:10:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
2,100012,2062,2589,2,0,,0,,0,,0,,0,0,,,0,0,0,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,0,11:22:00,11:40:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
3,100012,2062,2589,3,0,,0,,0,,0,,0,0,,,0,0,0,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,0,,0,0,0,12:00:00,14:00:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
4,100012,2062,2589,4,0,,0,,0,,0,,0,0,,,0,0,0,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,0,14:20:00,NaN,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284


In [40]:
# Add origin information from the previous row.
dic_read_last = {
    'o_act':        'd_act',
    'o_place_type': 'd_place_type',
    'o_place_name': 'd_place_name',
    'o_address':    'd_address',
    'o_lon':        'd_lon',
    'o_lat':        'd_lat',
    'o_taz':        'd_taz',
    'o_dep_time':   'next_trip_dep_time',
    'travel_date':  'travel_date'
}

for k, v in dic_read_last.items():
    df_temp.loc[df_temp['trip_number']!=0, k] = df_temp[v].shift(1)

df_temp['o_taz'] = df_temp['o_taz'].astype('int64')

df_temp.head()

,company_id,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,o_dep_time,d_arr_time,next_trip_dep_time,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz
0,100012,2062,2589,0,0,,0,,0,,0,,0.0,0.0,,,0.000000,0.000000,0,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,0,NaN,10:00:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
1,100012,2062,2589,1,0,,0,,0,,0,,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,0,,0,0,10:00:00,10:20:00,11:10:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
2,100012,2062,2589,2,0,,0,,0,,0,,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,11:10:00,11:22:00,11:40:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
3,100012,2062,2589,3,0,,0,,0,,0,,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,0,,0,0,11:40:00,12:00:00,14:00:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
4,100012,2062,2589,4,0,,0,,0,,0,,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,14:00:00,14:20:00,NaN,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284


In [41]:
# Remove the first record of each vehicle as it doesn't represent a trip, rather the 
# initial origin of the vehicle, whose critical info has already been stored in the next record.
df_temp = df_temp.loc[df_temp['trip_number']!=0]
df_temp.head()

,company_id,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,o_dep_time,d_arr_time,next_trip_dep_time,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz
1,100012,2062,2589,1,0,,0,,0,,0,,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,0,,0,0,10:00:00,10:20:00,11:10:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
2,100012,2062,2589,2,0,,0,,0,,0,,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,11:10:00,11:22:00,11:40:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
3,100012,2062,2589,3,0,,0,,0,,0,,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,0,,0,0,11:40:00,12:00:00,14:00:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
4,100012,2062,2589,4,0,,0,,0,,0,,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,14:00:00,14:20:00,NaN,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
6,100012,2063,2590,1,0,,0,,0,,0,,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,Soto St,-117.237975,32.752893,828,0,,0,0,14:00:00,14:11:00,16:45:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284


In [42]:
# Fill the activity and place segment fields.
df_temp['o_act_seg']      = df_temp['o_act'].map(lambda x: dic_at[x][0])
df_temp['o_act_seg_name'] = df_temp['o_act'].map(lambda x: dic_at[x][1])
df_temp['d_act_seg']      = df_temp['d_act'].apply(lambda x: dic_at[x][0])
df_temp['d_act_seg_name'] = df_temp['d_act'].apply(lambda x: dic_at[x][1])

df_temp['o_plc_seg']      = df_temp['o_place_type'].map(lambda x: dic_pt[x][0])
df_temp['o_plc_seg_name'] = df_temp['o_place_type'].map(lambda x: dic_pt[x][1])
df_temp['d_plc_seg']      = df_temp['d_place_type'].apply(lambda x: dic_pt[x][0])
df_temp['d_plc_seg_name'] = df_temp['d_place_type'].apply(lambda x: dic_pt[x][1])

df_temp.head()

,company_id,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,o_dep_time,d_arr_time,next_trip_dep_time,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz
1,100012,2062,2589,1,B,Base,1,Residential,M,Maintenance/Other,5,Retail and Restaurant,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,0,,0,0,10:00:00,10:20:00,11:10:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
2,100012,2062,2589,2,M,Maintenance/Other,5,Retail and Restaurant,B,Base,1,Residential,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,11:10:00,11:22:00,11:40:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
3,100012,2062,2589,3,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,0,,0,0,11:40:00,12:00:00,14:00:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
4,100012,2062,2589,4,S,Service,4,Other,B,Base,1,Residential,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,14:00:00,14:20:00,NaN,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284
6,100012,2063,2590,1,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,Soto St,-117.237975,32.752893,828,0,,0,0,14:00:00,14:11:00,16:45:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284


In [43]:
# df_temp.loc[(df_temp['d_act_seg']=='B')&(df_temp['d_taz']!=df_temp['estab_taz'])].head(1000)

In [44]:
df_temp.groupby(['d_act_seg', 'd_act_seg_name']).size()

d_act_seg  d_act_seg_name   
B          Base                 1621
D          Goods_Delivery       1948
H          Home                  392
M          Maintenance/Other    2317
P          Goods_Pickup          224
S          Service              4185
dtype: int64

##### Fix Activity Type 'Base' If Stop's TAZ Is Not Equal to Establishment TAZ

In [45]:
df_temp.loc[(df_temp['o_act_seg']=='B')&
            (df_temp['o_taz']!=df_temp['estab_taz'])&
            (df_temp['o_act'].isin([12])), ['o_act_seg', 'o_act_seg_name']
           ] = 'P', 'Goods_Pickup'

df_temp.loc[(df_temp['o_act_seg']=='B')&
            (df_temp['o_taz']!=df_temp['estab_taz'])&
            (df_temp['o_act'].isin([13])), ['o_act_seg', 'o_act_seg_name']
           ] = 'D', 'Goods_Delivery'

df_temp.loc[(df_temp['o_act_seg']=='B')&
            (df_temp['o_taz']!=df_temp['estab_taz'])&
            (~df_temp['o_act'].isin([12, 13])), ['o_act_seg', 'o_act_seg_name']
           ] = 'M', 'Maintenance/Other'

df_temp.loc[(df_temp['d_act_seg']=='B')&
            (df_temp['d_taz']!=df_temp['estab_taz'])&
            (df_temp['d_act'].isin([12])), ['d_act_seg', 'd_act_seg_name']
           ] = 'P', 'Goods_Pickup'

df_temp.loc[(df_temp['d_act_seg']=='B')&
            (df_temp['d_taz']!=df_temp['estab_taz'])&
            (df_temp['d_act'].isin([13])), ['d_act_seg', 'd_act_seg_name']
           ] = 'D', 'Goods_Delivery'

df_temp.loc[(df_temp['d_act_seg']=='B')&
            (df_temp['d_taz']!=df_temp['estab_taz'])&
            (~df_temp['d_act'].isin([12, 13])), ['d_act_seg', 'd_act_seg_name']
           ] = 'M', 'Maintenance/Other'

In [46]:
# df_temp.loc[df_temp['d_taz']!=df_temp['estab_taz']].head(1000)

In [47]:
df_temp.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10687 entries, 1 to 12260
Data columns (total 42 columns):
company_id            10687 non-null int64
vehicle_id            10687 non-null int64
driver_id             10687 non-null int64
trip_number           10687 non-null int64
o_act_seg             10687 non-null object
o_act_seg_name        10687 non-null object
o_plc_seg             10687 non-null int64
o_plc_seg_name        10687 non-null object
d_act_seg             10687 non-null object
d_act_seg_name        10687 non-null object
d_plc_seg             10687 non-null int64
d_plc_seg_name        10687 non-null object
o_act                 10687 non-null float64
o_place_type          10687 non-null float64
o_place_name          10687 non-null object
o_address             10687 non-null object
o_lon                 10687 non-null float64
o_lat                 10687 non-null float64
o_taz                 10687 non-null int64
d_act                 10687 non-null int64
d_place_type   

##### Include the industry of establishments

In [48]:
if dataset == 'CV':
    df_temp = df_temp.merge(df_estab[['company_id', 'base_location_Industry Group']], how='left', on='company_id')
    df_temp.rename(columns={'base_location_Industry Group': 'industry_code'}, inplace=True)
    df_temp = df_temp.merge(lookup_ind[['industry_code', 'industry_group']], how='left', on='industry_code')
    temp = df_temp.pop('industry_group')
    df_temp.insert(1, 'industry_group', temp) 

In [49]:
def omit_spaces(s):
    return s.replace(' ', '')

if dataset == 'TNC':
    df_temp = df_temp.merge(df_estab[['company_id', 'company_name']], how='left', on='company_id')
    
    # Some of the company names have leading or lagging spaces in their names.
    df_temp['company_name'] = df_temp['company_name'].apply(str.strip).apply(str.lower).apply(omit_spaces)
    df_temp['company_name'] = df_temp['company_name'].map(dic_tnc_names)
        
    # Figure out the industry groups.
    df_temp.insert(1, 'industry_group', "")
    df_temp['industry_group'] = df_temp['company_name'].map(dic_tnc_categories)
    
    df_temp['industry_group'] = 'TNC_' + df_temp['industry_group']

In [50]:
df_temp.head()

,company_id,industry_group,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,o_dep_time,d_arr_time,next_trip_dep_time,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz,industry_code
0,100012,Education/Other public services,2062,2589,1,B,Base,1,Residential,M,Maintenance/Other,5,Retail and Restaurant,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,0,,0,0,10:00:00,10:20:00,11:10:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9
1,100012,Education/Other public services,2062,2589,2,M,Maintenance/Other,5,Retail and Restaurant,B,Base,1,Residential,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,11:10:00,11:22:00,11:40:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9
2,100012,Education/Other public services,2062,2589,3,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,0,,0,0,11:40:00,12:00:00,14:00:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9
3,100012,Education/Other public services,2062,2589,4,S,Service,4,Other,B,Base,1,Residential,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,14:00:00,14:20:00,NaN,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9
4,100012,Education/Other public services,2063,2590,1,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,Soto St,-117.237975,32.752893,828,0,,0,0,14:00:00,14:11:00,16:45:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9


In [51]:
df_trips = df_temp.copy()
del df_temp
# gc.collect()

##### Clean the time fields

Glossary:<br>
|Field          | Meaning|
|:---------------|:--------------------|
|str_td | Travel Date as String | 
|str_o_dt|      Departure Time from the Origin as String|
|str_d_at |     Arrival Time at the Destination as String|
|str_nt_dt |    Next Trip Departure Time as String|
|o_dt       |   Departure Time from the Origin as TimeStamp|
|d_at        |  Arrival Time at the Destination as TimeStamp|
|nt_dt        | Next Trip Departure Time as TimeStamp|

In [52]:
print(df_trips['travel_date'].dtypes)
print(df_trips['o_dep_time'].dtypes)
print(df_trips['d_arr_time'].dtypes)
print(df_trips['next_trip_dep_time'].dtypes)

datetime64[ns]
object
object
object


In [53]:
df_trips['str_td'] = df_trips['travel_date'].dt.strftime('%Y-%m-%d')    # str_td = travel date as string
df_trips.head()

,company_id,industry_group,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,o_dep_time,d_arr_time,next_trip_dep_time,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz,industry_code,str_td
0,100012,Education/Other public services,2062,2589,1,B,Base,1,Residential,M,Maintenance/Other,5,Retail and Restaurant,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,0,,0,0,10:00:00,10:20:00,11:10:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30
1,100012,Education/Other public services,2062,2589,2,M,Maintenance/Other,5,Retail and Restaurant,B,Base,1,Residential,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,11:10:00,11:22:00,11:40:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30
2,100012,Education/Other public services,2062,2589,3,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,0,,0,0,11:40:00,12:00:00,14:00:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30
3,100012,Education/Other public services,2062,2589,4,S,Service,4,Other,B,Base,1,Residential,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,14:00:00,14:20:00,NaN,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30
4,100012,Education/Other public services,2063,2590,1,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,Soto St,-117.237975,32.752893,828,0,,0,0,14:00:00,14:11:00,16:45:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30


In [54]:
df_trips['o_dep_time'].describe()

count        10687
unique        1884
top       11:00:00
freq            73
Name: o_dep_time, dtype: object

In [55]:
if dataset == 'CV':
    df_trips['str_o_dt'] =  df_trips['o_dep_time'].astype(str)
    print(f"Max length of column str_o_dt: {max(df_trips['str_o_dt'].str.len())}")
    print(f"Min length of column str_o_dt: {min(df_trips['str_o_dt'].str.len())}\n")

    df_trips['str_d_at'] =  df_trips['d_arr_time'].astype(str)
    print(f"Max length of column str_d_at: {max(df_trips['str_d_at'].str.len())}")
    print(f"Min length of column str_d_at: {min(df_trips['str_d_at'].str.len())}")
    df_trips.loc[df_trips['str_d_at'].str.len()>8]
    df_trips['str_d_at'] =df_trips['str_d_at'].str[-8:]
    print(f"Max length of column str_d_at: {max(df_trips['str_d_at'].str.len())}, after cleaning")
    print(f"Min length of column str_d_at: {min(df_trips['str_d_at'].str.len())}, after cleaning\n")

    df_trips['str_nt_dt'] =  df_trips['next_trip_dep_time'].astype(str)
    print(f"Max length of column str_nt_dt: {max(df_trips['str_nt_dt'].str.len())}")
    print(f"Min length of column str_nt_dt: {min(df_trips['str_nt_dt'].str.len())}")
    df_trips.loc[df_trips['str_nt_dt'].str.len()<8].head(2)

Max length of column str_o_dt: 8
Min length of column str_o_dt: 8

Max length of column str_d_at: 19
Min length of column str_d_at: 8
Max length of column str_d_at: 8, after cleaning
Min length of column str_d_at: 8, after cleaning

Max length of column str_nt_dt: 8
Min length of column str_nt_dt: 3


In [56]:
if dataset == 'TNC':
    df_trips['str_o_dt'] =  df_trips['o_dep_time'].astype(str)
#     print('Before:\n', df_trips['str_o_dt'].loc[~df_trips['str_o_dt'].str.len().isin([8, 15])])
    df_trips['str_o_dt'] = df_trips['str_o_dt'].\
    apply(lambda x: x[11:] if x[:10]=='1900-01-01' else x)
#     print('\nAfter:\n', df_trips['str_o_dt'].loc[~df_trips['str_o_dt'].str.len().isin([8, 15])])
    df_trips['str_o_dt'].loc[df_trips['str_o_dt'].str.len()!=8]
    df_trips['str_o_dt'].loc[df_trips['str_o_dt'].str.len()==8]
    df_trips.loc[df_trips['str_o_dt'].str.len()!=8, 'str_o_dt'] = df_trips['str_o_dt'].str[:-7]
    print(f"\n'o_dt' data are now clean: " +
          f"{len(df_trips.loc[df_trips['str_o_dt'].str.len()==8]) == len(df_trips)}\n\n")
    
    df_trips['str_d_at'] =  df_trips['d_arr_time'].astype(str)
#     print('Before:\n', df_trips['str_d_at'].loc[~df_trips['str_d_at'].str.len().isin([8, 15])])
    df_trips['str_d_at'] = df_trips['str_d_at'].\
    apply(lambda x: x[11:] if x[:10]=='1900-01-01' else x)
#     print('\nAfter:\n', df_trips['str_d_at'].loc[~df_trips['str_d_at'].str.len().isin([8, 15])])
    df_trips['str_d_at'].loc[df_trips['str_d_at'].str.len()!=8]
    df_trips['str_d_at'].loc[df_trips['str_d_at'].str.len()==8]
    df_trips.loc[df_trips['str_d_at'].str.len()!=8, 'str_d_at'] = df_trips['str_d_at'].str[:-7]
    print(f"\n'd_at' data are now clean: " +
          f"{len(df_trips.loc[df_trips['str_d_at'].str.len()==8]) == len(df_trips)}")

    df_trips['str_nt_dt'] =  df_trips['next_trip_dep_time'].astype(str)
#     print('Before:\n', df_trips['str_nt_dt'].loc[~df_trips['str_nt_dt'].str.len().isin([3, 8, 15])])
    df_trips['str_nt_dt'] = df_trips['str_nt_dt'].\
    apply(lambda x: x[11:] if x[:10]=='1900-01-01' else x)
#     print('\nAfter:\n', df_trips['str_nt_dt'].loc[~df_trips['str_nt_dt'].str.len().isin([3, 8, 15])])
    df_trips['str_nt_dt'].loc[~df_trips['str_nt_dt'].str.len().isin([3, 8])]
    df_trips['str_nt_dt'].loc[df_trips['str_nt_dt'].str.len()==8]
    df_trips.loc[df_trips['str_nt_dt'].str.len()==15, 'str_nt_dt'] = df_trips['str_nt_dt'].str[:-7]
    actual_times = len(df_trips.loc[df_trips['str_nt_dt'].str.len()==8])
    nans = len(df_trips.loc[df_trips['str_nt_dt'].str.len()==3])    
    print(f"\n'nt_dt' data are now clean: " +
          f"{actual_times+nans==len(df_trips)}\n")
    print(f'Number of recorded times: {actual_times}')
    print(f'Number of NANs: {nans}')
    print(f'Total trips in the dataframe: {len(df_trips)}')

In [57]:
df_trips['o_dt'] = df_trips['str_td'] + ' ' + df_trips['str_o_dt']
df_trips['o_dt'] = pd.to_datetime(df_trips['o_dt'])
print(type(df_trips['o_dt']))
print(type(df_trips['o_dt'][0]))
df_trips.head()

<class 'pandas.core.series.Series'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


,company_id,industry_group,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,o_dep_time,d_arr_time,next_trip_dep_time,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz,industry_code,str_td,str_o_dt,str_d_at,str_nt_dt,o_dt
0,100012,Education/Other public services,2062,2589,1,B,Base,1,Residential,M,Maintenance/Other,5,Retail and Restaurant,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,0,,0,0,10:00:00,10:20:00,11:10:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,10:00:00,10:20:00,11:10:00,2022-09-30 10:00:00
1,100012,Education/Other public services,2062,2589,2,M,Maintenance/Other,5,Retail and Restaurant,B,Base,1,Residential,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,11:10:00,11:22:00,11:40:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,11:10:00,11:22:00,11:40:00,2022-09-30 11:10:00
2,100012,Education/Other public services,2062,2589,3,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,0,,0,0,11:40:00,12:00:00,14:00:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,11:40:00,12:00:00,14:00:00,2022-09-30 11:40:00
3,100012,Education/Other public services,2062,2589,4,S,Service,4,Other,B,Base,1,Residential,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,14:00:00,14:20:00,NaN,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,14:00:00,14:20:00,nan,2022-09-30 14:00:00
4,100012,Education/Other public services,2063,2590,1,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,Soto St,-117.237975,32.752893,828,0,,0,0,14:00:00,14:11:00,16:45:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,14:00:00,14:11:00,16:45:00,2022-09-30 14:00:00


In [58]:
df_trips['d_at'] = df_trips['str_td'] + ' ' + df_trips['str_d_at']
df_trips['d_at'] = pd.to_datetime(df_trips['d_at'])
print(type(df_trips['d_at']))
print(type(df_trips['d_at'][0]))
print(f"Number of NAs in this column: {df_trips['d_at'].isna().sum()}")
df_trips.head()

<class 'pandas.core.series.Series'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Number of NAs in this column: 0


,company_id,industry_group,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,o_dep_time,d_arr_time,next_trip_dep_time,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz,industry_code,str_td,str_o_dt,str_d_at,str_nt_dt,o_dt,d_at
0,100012,Education/Other public services,2062,2589,1,B,Base,1,Residential,M,Maintenance/Other,5,Retail and Restaurant,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,0,,0,0,10:00:00,10:20:00,11:10:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,10:00:00,10:20:00,11:10:00,2022-09-30 10:00:00,2022-09-30 10:20:00
1,100012,Education/Other public services,2062,2589,2,M,Maintenance/Other,5,Retail and Restaurant,B,Base,1,Residential,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,11:10:00,11:22:00,11:40:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,11:10:00,11:22:00,11:40:00,2022-09-30 11:10:00,2022-09-30 11:22:00
2,100012,Education/Other public services,2062,2589,3,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,0,,0,0,11:40:00,12:00:00,14:00:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,11:40:00,12:00:00,14:00:00,2022-09-30 11:40:00,2022-09-30 12:00:00
3,100012,Education/Other public services,2062,2589,4,S,Service,4,Other,B,Base,1,Residential,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,14:00:00,14:20:00,NaN,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,14:00:00,14:20:00,nan,2022-09-30 14:00:00,2022-09-30 14:20:00
4,100012,Education/Other public services,2063,2590,1,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,Soto St,-117.237975,32.752893,828,0,,0,0,14:00:00,14:11:00,16:45:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,14:00:00,14:11:00,16:45:00,2022-09-30 14:00:00,2022-09-30 14:11:00


In [59]:
df_trips['nt_dt'] = df_trips['str_td'] + ' ' + df_trips['str_nt_dt']
df_trips['nt_dt'] = pd.to_datetime(df_trips['nt_dt'], errors='coerce')
print(type(df_trips['nt_dt']))
print(type(df_trips['nt_dt'][0]))
print(f"Number of NaTs in this column: {df_trips['nt_dt'].isna().sum()}")
df_trips.head()

<class 'pandas.core.series.Series'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Number of NaTs in this column: 1574


,company_id,industry_group,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,o_dep_time,d_arr_time,next_trip_dep_time,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz,industry_code,str_td,str_o_dt,str_d_at,str_nt_dt,o_dt,d_at,nt_dt
0,100012,Education/Other public services,2062,2589,1,B,Base,1,Residential,M,Maintenance/Other,5,Retail and Restaurant,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,0,,0,0,10:00:00,10:20:00,11:10:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,10:00:00,10:20:00,11:10:00,2022-09-30 10:00:00,2022-09-30 10:20:00,2022-09-30 11:10:00
1,100012,Education/Other public services,2062,2589,2,M,Maintenance/Other,5,Retail and Restaurant,B,Base,1,Residential,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,11:10:00,11:22:00,11:40:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,11:10:00,11:22:00,11:40:00,2022-09-30 11:10:00,2022-09-30 11:22:00,2022-09-30 11:40:00
2,100012,Education/Other public services,2062,2589,3,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,0,,0,0,11:40:00,12:00:00,14:00:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,11:40:00,12:00:00,14:00:00,2022-09-30 11:40:00,2022-09-30 12:00:00,2022-09-30 14:00:00
3,100012,Education/Other public services,2062,2589,4,S,Service,4,Other,B,Base,1,Residential,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,14:00:00,14:20:00,NaN,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,14:00:00,14:20:00,nan,2022-09-30 14:00:00,2022-09-30 14:20:00,NaT
4,100012,Education/Other public services,2063,2590,1,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,Soto St,-117.237975,32.752893,828,0,,0,0,14:00:00,14:11:00,16:45:00,0,0,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,14:00:00,14:11:00,16:45:00,2022-09-30 14:00:00,2022-09-30 14:11:00,2022-09-30 16:45:00


In [60]:
# Some tests:
print(df_trips['d_at'][0] - df_trips['o_dt'][0])
print(df_trips['d_at'][0] > df_trips['o_dt'][0])
print(df_trips['nt_dt'][3] - df_trips['d_at'][3])
print(df_trips['nt_dt'][0] + pd.to_timedelta(1, unit='D'))

0 days 00:20:00
True
NaT
2022-10-01 11:10:00


In [61]:
# Ensure events are in chronological order.
print(len(df_trips.loc[df_trips['d_at']<df_trips['o_dt']]))
df_trips.loc[df_trips['d_at']<df_trips['o_dt'], 'd_at'] = df_trips['d_at'] + pd.to_timedelta(1, unit='days')
print(len(df_trips.loc[df_trips['d_at']<df_trips['o_dt']]), '\n')

print(len(df_trips.loc[df_trips['nt_dt']<df_trips['d_at']]))
df_trips.loc[df_trips['nt_dt']<df_trips['d_at'], 'nt_dt'] = df_trips['nt_dt'] + pd.to_timedelta(1, unit='days')
print(len(df_trips.loc[df_trips['nt_dt']<df_trips['d_at']]))

11
0 

0
0


In [62]:
# Find travel time and stop duration in minutes.
# df_trips['travel_time'] = ((df_trips['d_at'] - df_trips['o_dt']).dt.seconds).astype(int)
# df_trips['stop_duration'] = ((df_trips['nt_dt'] - df_trips['d_at']).dt.seconds).round().astype('Int32') #, errors='ignore'
df_trips['travel_time'] = (((df_trips['d_at'] - df_trips['o_dt']).dt.seconds) / 60).astype(int)
df_trips['stop_duration'] = (((df_trips['nt_dt'] - df_trips['d_at']).dt.seconds) / 60).round().astype('Int32') #, errors='ignore'
# Note the capitalized 'Int32' in the line above - as opposed to 'int32'.
# print(f"Now, the datatype for the 'stop_duration' column is kind of weird: {df_trips['stop_duration'].dtypes}, rather than simply int32.")
# print('We need to live with this because we want to allow <NA> values in this column.')
df_trips.head()

,company_id,industry_group,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,o_dep_time,d_arr_time,next_trip_dep_time,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz,industry_code,str_td,str_o_dt,str_d_at,str_nt_dt,o_dt,d_at,nt_dt
0,100012,Education/Other public services,2062,2589,1,B,Base,1,Residential,M,Maintenance/Other,5,Retail and Restaurant,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,0,,0,0,10:00:00,10:20:00,11:10:00,20,50,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,10:00:00,10:20:00,11:10:00,2022-09-30 10:00:00,2022-09-30 10:20:00,2022-09-30 11:10:00
1,100012,Education/Other public services,2062,2589,2,M,Maintenance/Other,5,Retail and Restaurant,B,Base,1,Residential,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,11:10:00,11:22:00,11:40:00,12,18,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,11:10:00,11:22:00,11:40:00,2022-09-30 11:10:00,2022-09-30 11:22:00,2022-09-30 11:40:00
2,100012,Education/Other public services,2062,2589,3,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,0,,0,0,11:40:00,12:00:00,14:00:00,20,120,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,11:40:00,12:00:00,14:00:00,2022-09-30 11:40:00,2022-09-30 12:00:00,2022-09-30 14:00:00
3,100012,Education/Other public services,2062,2589,4,S,Service,4,Other,B,Base,1,Residential,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,14:00:00,14:20:00,NaN,20,NaN,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,14:00:00,14:20:00,nan,2022-09-30 14:00:00,2022-09-30 14:20:00,NaT
4,100012,Education/Other public services,2063,2590,1,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,Soto St,-117.237975,32.752893,828,0,,0,0,14:00:00,14:11:00,16:45:00,11,154,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30,14:00:00,14:11:00,16:45:00,2022-09-30 14:00:00,2022-09-30 14:11:00,2022-09-30 16:45:00


In [63]:
# Throw away unnecessary time fields.
rmv_fields = ['o_dep_time', 'd_arr_time', 'next_trip_dep_time',
              'str_td', 'str_o_dt', 'str_d_at', 'str_nt_dt']
all_fields = df_trips.columns.to_list()
keep_cols = [f for f in all_fields if not f in rmv_fields]        

df_trips = df_trips[keep_cols]
df_trips.head()

,company_id,industry_group,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz,industry_code,o_dt,d_at,nt_dt
0,100012,Education/Other public services,2062,2589,1,B,Base,1,Residential,M,Maintenance/Other,5,Retail and Restaurant,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,0,,0,0,20,50,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 10:00:00,2022-09-30 10:20:00,2022-09-30 11:10:00
1,100012,Education/Other public services,2062,2589,2,M,Maintenance/Other,5,Retail and Restaurant,B,Base,1,Residential,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,12,18,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 11:10:00,2022-09-30 11:22:00,2022-09-30 11:40:00
2,100012,Education/Other public services,2062,2589,3,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,0,,0,0,20,120,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 11:40:00,2022-09-30 12:00:00,2022-09-30 14:00:00
3,100012,Education/Other public services,2062,2589,4,S,Service,4,Other,B,Base,1,Residential,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,20,NaN,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 14:00:00,2022-09-30 14:20:00,NaT
4,100012,Education/Other public services,2063,2590,1,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,Soto St,-117.237975,32.752893,828,0,,0,0,11,154,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 14:00:00,2022-09-30 14:11:00,2022-09-30 16:45:00


##### Mark the last trips

In [64]:
df_trips['last_trip'] = False
df_trips.loc[df_trips['trip_number'].shift(-1)==1, 'last_trip'] = True
df_trips.at[df_trips.index[-1], 'last_trip'] = True
df_trips.tail()
df_trips.head(50)

,company_id,industry_group,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz,industry_code,o_dt,d_at,nt_dt,last_trip
0,100012,Education/Other public services,2062,2589,1,B,Base,1,Residential,M,Maintenance/Other,5,Retail and Restaurant,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,0,,0,0,20,50,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 10:00:00,2022-09-30 10:20:00,2022-09-30 11:10:00,False
1,100012,Education/Other public services,2062,2589,2,M,Maintenance/Other,5,Retail and Restaurant,B,Base,1,Residential,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,12,18,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 11:10:00,2022-09-30 11:22:00,2022-09-30 11:40:00,False
2,100012,Education/Other public services,2062,2589,3,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,0,,0,0,20,120,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 11:40:00,2022-09-30 12:00:00,2022-09-30 14:00:00,False
3,100012,Education/Other public services,2062,2589,4,S,Service,4,Other,B,Base,1,Residential,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,20,NaN,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 14:00:00,2022-09-30 14:20:00,NaT,True
4,100012,Education/Other public services,2063,2590,1,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,Soto St,-117.237975,32.752893,828,0,,0,0,11,154,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 14:00:00,2022-09-30 14:11:00,2022-09-30 16:45:00,False
5,100012,Education/Other public services,2063,2590,2,S,Service,4,Other,M,Maintenance/Other,6,Gas,10.0,96.0,DOG PARK,Soto St,-117.237975,32.752893,828,2,16,FUEL STOP,4201 W Point Loma Blvd,-117.226801,32.752382,1010,0,,0,0,2,23,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 16:45:00,2022-09-30 16:47:00,2022-09-30 17:10:00,False
6,100012,Education/Other public services,2063,2590,3,M,Maintenance/Other,6,Gas,B,Base,1,Residential,2.0,16.0,FUEL STOP,4201 W Point Loma Blvd,-117.226801,32.752382,1010,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,12,NaN,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 17:10:00,2022-09-30 17:22:00,NaT,True
7,100012,Education/Other public services,2064,2591,1,B,Base,1,Residential,S,Service,2,Office,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,1,PACIFIC COAST ANIMAL CONTROL,3812 Sierra Morena Ave,-117.314980,33.163175,193,0,,0,0,38,112,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 10:00:00,2022-09-30 10:38:00,2022-09-30 12:30:00,False
8,100012,Education/Other public services,2064,2591,2,S,Service,2,Office,B,Base,1,Residential,10.0,1.0,PACIFIC COAST ANIMAL CONTROL,3812 Sierra Morena Ave,-117.314980,33.163175,193,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,0,,0,0,68,22,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 12:30:00,2022-09-30 13:38:00,2022-09-30 14:00:00,False
9,100012,Education/Other public services,2064,2591,3,B,Base,1,Residential,S,Service,2,Office,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,8,VET,2675 B St,-117.136609,32.717661,2252,0,,0,0,1,19,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 14:00:00,2022-09-30 14:01:00,2022-09-30 14:20:00,False


##### Find the trip distance

In [65]:
df_trips['tod'] = ""

In [66]:
# Indicate at what time of day the trip has started.
def find_tod(timestamp):
    if timestamp.time() < dt.time(3):
        return 'EV' # Late: 7PM to 3AM
    elif timestamp.time() < dt.time(6):
        return 'EA' # Early: 3AM to 6AM
    elif timestamp.time() < dt.time(9):
        return 'AM' # AM Peak: 6AM to 9AM
    elif timestamp.time() < dt.time(15, 30):
        return 'MD' # Midday: 9AM to 3:30PM
    elif timestamp.time() < dt.time(19):
        return 'PM' # PM Peak: 3:30PM to 7pm
    elif timestamp.time() >= dt.time(19):
        return 'EV' # Late: 7PM to 3AM

df_trips['tod'] = df_trips['o_dt'].apply(find_tod) 
# df_trips[['o_dt', 'tod']].loc[(df_trips['o_dt'].dt.time>=dt.time(0)) & (df_trips['o_dt'].dt.time<dt.time(3))].head(10)
# df_trips[['o_dt', 'tod']].loc[(df_trips['o_dt'].dt.time>=dt.time(3)) & (df_trips['o_dt'].dt.time<dt.time(6))].head(10)
# df_trips[['o_dt', 'tod']].loc[(df_trips['o_dt'].dt.time>=dt.time(6)) & (df_trips['o_dt'].dt.time<dt.time(9))].head(10)
# df_trips[['o_dt', 'tod']].loc[(df_trips['o_dt'].dt.time>=dt.time(9)) & (df_trips['o_dt'].dt.time<dt.time(15, 30))].head(10)
# df_trips[['o_dt', 'tod']].loc[(df_trips['o_dt'].dt.time>=dt.time(15,30)) & (df_trips['o_dt'].dt.time<dt.time(19))].head(10)
df_trips[['o_dt', 'tod']].loc[(df_trips['o_dt'].dt.time>=dt.time(19)) & (df_trips['o_dt'].dt.time<dt.time(23, 59, 59, 999999))].head(10)

,o_dt,tod
238,2022-09-15 19:10:00,EV
239,2022-09-15 20:03:00,EV
266,2022-09-02 19:15:44,EV
267,2022-09-02 19:24:42,EV
760,2022-07-19 20:26:00,EV
761,2022-07-19 20:42:00,EV
772,2022-08-30 22:10:00,EV
773,2022-08-30 22:30:00,EV
774,2022-08-30 22:35:00,EV
775,2022-08-30 22:55:00,EV


In [67]:
print(f'Example: Distance from TAZ 5 to TAZ 19 in AM peak for medium trucks is: {dist_dfs[1][1].at[4,18]/10:} miles')

Example: Distance from TAZ 5 to TAZ 19 in AM peak for medium trucks is: 103.2 miles


In [68]:
dic_v = {'LCV': 0, 'SUT': 1, 'MUT': 2}
dic_tod = {'EA': 0, 'AM': 1, 'MD':2, 'PM':3, 'EV':4}

def find_dist(o_taz, d_taz, vs, tod): # vs is vehicle size and tod is time of day.
    if o_taz==-1 or d_taz==-1: return None
    return dist_dfs[tod][vs].at[o_taz-1, d_taz-1] / 10

df_trips['trip_dist'] = df_trips.apply(lambda x: find_dist(x['o_taz'], x['d_taz'],
                                                           dic_v[x['veh_type']],
                                                           dic_tod[x['tod']]), axis=1)
    
df_trips.head()

,company_id,industry_group,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz,industry_code,o_dt,d_at,nt_dt,last_trip,tod
0,100012,Education/Other public services,2062,2589,1,B,Base,1,Residential,M,Maintenance/Other,5,Retail and Restaurant,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,5.6,,0,0,20,50,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 10:00:00,2022-09-30 10:20:00,2022-09-30 11:10:00,False,MD
1,100012,Education/Other public services,2062,2589,2,M,Maintenance/Other,5,Retail and Restaurant,B,Base,1,Residential,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,6.9,,0,0,12,18,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 11:10:00,2022-09-30 11:22:00,2022-09-30 11:40:00,False,MD
2,100012,Education/Other public services,2062,2589,3,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,2.8,,0,0,20,120,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 11:40:00,2022-09-30 12:00:00,2022-09-30 14:00:00,False,MD
3,100012,Education/Other public services,2062,2589,4,S,Service,4,Other,B,Base,1,Residential,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,3.2,,0,0,20,NaN,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 14:00:00,2022-09-30 14:20:00,NaT,True,MD
4,100012,Education/Other public services,2063,2590,1,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,Soto St,-117.237975,32.752893,828,8.1,,0,0,11,154,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 14:00:00,2022-09-30 14:11:00,2022-09-30 16:45:00,False,MD


In [69]:
df_trips[['o_taz','d_taz','veh_type','tod']].head()

,o_taz,d_taz,veh_type,tod
0,2284,2091,LCV,MD
1,2091,2284,LCV,MD
2,2284,1933,LCV,MD
3,1933,2284,LCV,MD
4,2284,828,LCV,MD


##### Find the trip generalized travel time from the establishment base to the trip destination

In [70]:
print(f'Example: Generalized travel time from TAZ 3901 to TAZ 4233 in mid-day for heavy trucks is: {g_tt_dfs[2][2].iloc[3900, 4232]} minutes.')

Example: Generalized travel time from TAZ 3901 to TAZ 4233 in mid-day for heavy trucks is: 16.15 minutes.


In [71]:
dic_v = {'LCV': 0, 'SUT': 1, 'MUT': 2}
for i, row in df_trips.iterrows():    
    if row['estab_taz'] == -1 or row['d_taz'] == -1: continue
    df_trips.at[i, 'toll_in_cents_from_base'] = toll_dfs[2][dic_v[row['veh_type']]].at[row['estab_taz']-1, row['d_taz']-1]
    df_trips.at[i, 'gen_tt_from_base'] = g_tt_dfs[2][dic_v[row['veh_type']]].at[row['estab_taz']-1, row['d_taz']-1]

In [72]:
out_file = f'Generalized TT for {dataset}.xlsx'
out_path = os.path.join(project_path, out_data_dir, out_file)
df_trips.to_excel(out_path, engine='openpyxl')

##### Calculate average generalized travel time by industry by destination activity

In [73]:
print(len(df_trips.loc[df_trips['gen_tt_from_base'].isnull()]))
df_trips.loc[df_trips['gen_tt_from_base'].isnull()].head(2)

322


,company_id,industry_group,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz,industry_code,o_dt,d_at,nt_dt,last_trip,tod,toll_in_cents_from_base,gen_tt_from_base
192,100279,Agriculture/Mining,337,659,2,M,Maintenance/Other,6,Gas,P,Goods_Pickup,3,Warehouse,3.0,16.0,ARCO GAS,810 Tamarack Ave,-117.337783,33.152044,127,6,14,ROXFORD PRODUCE INTERNATIONAL,13287 Ralston Ave,-118.473418,34.312105,-1,NaN,,0,0,129,59,LCV,1.0,NaN,2022-08-02,smartphone,9.375,279,1,2022-08-02 04:42:00,2022-08-02 06:51:00,2022-08-02 07:50:00,False,EA,NaN,NaN
193,100279,Agriculture/Mining,337,659,3,P,Goods_Pickup,3,Warehouse,M,Maintenance/Other,6,Gas,6.0,14.0,ROXFORD PRODUCE INTERNATIONAL,13287 Ralston Ave,-118.473418,34.312105,-1,2,16,GAS STATION,15948 Roxford St,-118.478316,34.303948,-1,NaN,,0,0,4,7,LCV,NaN,NaN,2022-08-02,smartphone,9.375,279,1,2022-08-02 07:50:00,2022-08-02 07:54:00,2022-08-02 08:01:00,False,AM,NaN,NaN


In [74]:
cols = ['industry_group', 'd_act_seg', 'd_plc_seg_name', 'veh_type', 'expnsn_factor', 'gen_tt_from_base']
df_gtt = df_trips[cols].copy()
df_gtt = df_gtt.loc[~df_gtt['gen_tt_from_base'].isnull()]
df_gtt = df_gtt.loc[df_gtt['d_act_seg'].isin(['P', 'D', 'S'])]
df_gtt.rename(columns={'d_act_seg': 'purpose'}, inplace=True)
df_gtt['purpose'] = df_gtt['purpose'].apply(lambda x: 'Service' if x=='S' else 'Goods')
# df_gtt.insert(4, 'count', 1)
df_gtt.insert(3, 'customer', 'Business')
df_gtt.loc[df_gtt['d_plc_seg_name']=='Residential', 'customer'] = 'Resident'
df_gtt.drop(columns='d_plc_seg_name', inplace=True)
df_gtt.head(2)

,industry_group,purpose,customer,veh_type,expnsn_factor,gen_tt_from_base
2,Education/Other public services,Service,Business,LCV,177.825038,8.77
4,Education/Other public services,Service,Business,LCV,177.825038,11.99


###### Unweighted generalized travel time

In [75]:
df_gtt_unw1 = df_gtt.groupby(['industry_group', 'purpose', 'customer', 'veh_type']).size().rename('sample_size').reset_index()
df_gtt_unw1.head()

,industry_group,purpose,customer,veh_type,sample_size
0,Agriculture/Mining,Goods,Business,LCV,15
1,Agriculture/Mining,Goods,Business,MUT,2
2,Agriculture/Mining,Goods,Resident,LCV,5
3,Agriculture/Mining,Service,Business,LCV,52
4,Agriculture/Mining,Service,Business,MUT,7


In [76]:
df_gtt_unw2 = df_gtt.groupby(['industry_group', 'purpose', 'customer', 'veh_type'])['gen_tt_from_base']\
.mean().reset_index()
df_gtt_unw2.head()

,industry_group,purpose,customer,veh_type,gen_tt_from_base
0,Agriculture/Mining,Goods,Business,LCV,25.017333
1,Agriculture/Mining,Goods,Business,MUT,28.535000
2,Agriculture/Mining,Goods,Resident,LCV,23.902000
3,Agriculture/Mining,Service,Business,LCV,29.152308
4,Agriculture/Mining,Service,Business,MUT,20.868571


In [77]:
df_gtt_unw = pd.merge(df_gtt_unw1, df_gtt_unw2, on=['industry_group', 'purpose', 'customer', 'veh_type']).dropna()
df_gtt_unw['product'] = df_gtt_unw['sample_size'] * df_gtt_unw['gen_tt_from_base']
df_gtt_unw.head()

,industry_group,purpose,customer,veh_type,sample_size,gen_tt_from_base,product
0,Agriculture/Mining,Goods,Business,LCV,15,25.017333,375.26
1,Agriculture/Mining,Goods,Business,MUT,2,28.535000,57.07
2,Agriculture/Mining,Goods,Resident,LCV,5,23.902000,119.51
3,Agriculture/Mining,Service,Business,LCV,52,29.152308,1515.92
4,Agriculture/Mining,Service,Business,MUT,7,20.868571,146.08


In [78]:
s = df_gtt_unw.groupby(['industry_group', 'purpose', 'customer'])['sample_size'].sum()
df_avg_gtt_unw = ((df_gtt_unw.groupby(['industry_group', 'purpose', 'customer'])['product'].sum()/s)\
                  .rename('avg_gen_tt_unweighted')).reset_index()
df_avg_gtt_unw = s.reset_index().merge(df_avg_gtt_unw, on=['industry_group', 'purpose', 'customer'] )
df_avg_gtt_unw.head()

,industry_group,purpose,customer,sample_size,avg_gen_tt_unweighted
0,Agriculture/Mining,Goods,Business,17,25.431176
1,Agriculture/Mining,Goods,Resident,5,23.902000
2,Agriculture/Mining,Service,Business,83,35.046867
3,Agriculture/Mining,Service,Resident,41,31.684878
4,Construction,Goods,Business,71,23.832817


In [79]:
out_file = f'Generalized TT for {dataset} Segments, Unweighted.xlsx'
out_path = os.path.join(project_path, out_data_dir, out_file)
df_avg_gtt_unw.to_excel(out_path, engine='openpyxl')

###### Weighted generalized travel time

In [80]:
df_gtt2 = df_gtt.copy()
df_gtt2['expanded_tt'] = df_gtt2.expnsn_factor * df_gtt2.gen_tt_from_base
df_gtt2.head(1)

,industry_group,purpose,customer,veh_type,expnsn_factor,gen_tt_from_base,expanded_tt
2,Education/Other public services,Service,Business,LCV,177.825038,8.77,1559.525583


In [81]:
df_gtt2_wei0 = df_gtt2.groupby(['industry_group', 'purpose', 'customer', 'veh_type']).size().rename('sample_size').reset_index()
df_gtt2_wei0.head()

,industry_group,purpose,customer,veh_type,sample_size
0,Agriculture/Mining,Goods,Business,LCV,15
1,Agriculture/Mining,Goods,Business,MUT,2
2,Agriculture/Mining,Goods,Resident,LCV,5
3,Agriculture/Mining,Service,Business,LCV,52
4,Agriculture/Mining,Service,Business,MUT,7


In [82]:
df_gtt2_wei1 = df_gtt2.groupby(['industry_group', 'purpose', 'customer', 'veh_type'])['expnsn_factor']\
.sum().rename('pop_size').reset_index()
df_gtt2_wei1.head()

,industry_group,purpose,customer,veh_type,pop_size
0,Agriculture/Mining,Goods,Business,LCV,151.640938
1,Agriculture/Mining,Goods,Business,MUT,713.092329
2,Agriculture/Mining,Goods,Resident,LCV,63.398907
3,Agriculture/Mining,Service,Business,LCV,564.611566
4,Agriculture/Mining,Service,Business,MUT,2495.823153


In [83]:
df_gtt2_wei2 = df_gtt2.groupby(['industry_group', 'purpose', 'customer', 'veh_type'])['expanded_tt']\
.sum().reset_index()
df_gtt2_wei2.head()

,industry_group,purpose,customer,veh_type,expanded_tt
0,Agriculture/Mining,Goods,Business,LCV,4013.614474
1,Agriculture/Mining,Goods,Business,MUT,20348.089620
2,Agriculture/Mining,Goods,Resident,LCV,1418.056897
3,Agriculture/Mining,Service,Business,LCV,15901.484740
4,Agriculture/Mining,Service,Business,MUT,52084.263740


In [84]:
df_gtt2_wei = pd.merge(df_gtt2_wei0, df_gtt2_wei1, on=['industry_group', 'purpose', 'customer', 'veh_type'])
df_gtt2_wei = pd.merge(df_gtt2_wei, df_gtt2_wei2, on=['industry_group', 'purpose', 'customer', 'veh_type']).dropna()
# df_gtt2_wei['product'] = df_gtt2_wei['pop_size'] * df_gtt2_wei['expanded_tt']
df_gtt2_wei.head()

,industry_group,purpose,customer,veh_type,sample_size,pop_size,expanded_tt
0,Agriculture/Mining,Goods,Business,LCV,15,151.640938,4013.614474
1,Agriculture/Mining,Goods,Business,MUT,2,713.092329,20348.089620
2,Agriculture/Mining,Goods,Resident,LCV,5,63.398907,1418.056897
3,Agriculture/Mining,Service,Business,LCV,52,564.611566,15901.484740
4,Agriculture/Mining,Service,Business,MUT,7,2495.823153,52084.263740


In [85]:
sz= df_gtt2_wei.groupby(['industry_group', 'purpose', 'customer'])['sample_size'].sum()
s = df_gtt2_wei.groupby(['industry_group', 'purpose', 'customer'])['pop_size'].sum()
df_avg_gtt2_wei = ((df_gtt2_wei.groupby(['industry_group', 'purpose', 'customer'])['expanded_tt'].sum()/s)\
                  .rename('avg_gen_tt_weighted')).reset_index()
s = sz.reset_index().merge(s, on=['industry_group', 'purpose', 'customer'] )
df_avg_gtt2_wei = s.reset_index().merge(df_avg_gtt2_wei, on=['industry_group', 'purpose', 'customer'] )
df_avg_gtt2_wei.head()

,index,industry_group,purpose,customer,sample_size,pop_size,avg_gen_tt_weighted
0,0,Agriculture/Mining,Goods,Business,17,864.733267,28.172507
1,1,Agriculture/Mining,Goods,Resident,5,63.398907,22.367214
2,2,Agriculture/Mining,Service,Business,83,3371.647834,24.959439
3,3,Agriculture/Mining,Service,Resident,41,1912.676007,27.081218
4,4,Construction,Goods,Business,71,13647.715972,25.655633


In [86]:
out_file = f'Generalized TT for {dataset} Segments, Weighted.xlsx'
out_path = os.path.join(project_path, out_data_dir, out_file)
df_avg_gtt2_wei.to_excel(out_path, engine='openpyxl')

#### Create Summaries from the Trip Dataframe

In [87]:
# Unweighted:
AvgStop1 = df_trips[df_trips['stop_duration']>0].groupby(['d_act_seg_name', 'industry_group']).size().rename('count')
AvgStop2 = df_trips[df_trips['stop_duration']>0].groupby(['d_act_seg_name', 'industry_group'])['stop_duration']\
.mean().rename('stop_duration_minutes').round()

# Weighted:
df_trips.loc[df_trips['stop_duration']>0, 'dur_by_weight'] = df_trips['stop_duration'] * df_trips['expnsn_factor']
AvgStop1w = df_trips[df_trips['stop_duration']>0].groupby(['d_act_seg_name', 'industry_group'])['expnsn_factor'].sum().rename('weighted_count')
AvgStop2w = (df_trips[df_trips['stop_duration']>0].groupby(['d_act_seg_name', 'industry_group'])['dur_by_weight'].sum()/\
            df_trips[df_trips['stop_duration']>0].groupby(['d_act_seg_name', 'industry_group'])['expnsn_factor'].sum()).\
            rename('stop_duration_minutes').round()

In [88]:
# df_trips[df_trips['stop_duration']>0][['stop_duration','expnsn_factor']].sort_values(['expnsn_factor'], ascending=True)

In [89]:
AvgStop = pd.concat([AvgStop1, AvgStop2], axis=1)
AvgStop_w = pd.concat([AvgStop1w, AvgStop2w], axis=1)
AvgStop
AvgStop_w

weighted_count  \
d_act_seg_name    industry_group                                                       
Base              Agriculture/Mining                                       41.092213   
                  Construction                                           9212.959715   
                  Education/Other public services                       22624.591639   
                  Industrial/Utilities                                   1409.956655   
                  Info/Finance/Insurance/Real Estate/Professional...    10798.237452   
                  Leisure/Accommodations and Food                        2099.181795   
                  Manufacturing                                          3753.003469   
                  Medical/Health Services                               17565.315462   
                  Retail                                                 4457.208138   
                  Transportation                                         3044.081924   
                  Wholesale                                              2056.656379   
Goods_Delivery    Agriculture/Mining                                      937.267876   
                  Construction                                          14471.344954   
                  Education/Other public services                        1731.687903   
                  Industrial/Utilities                                    727.300182   
                  Info/Finance/Insurance/Real Estate/Professional...     1237.479642   
                  Leisure/Accommodations and Food                        1534.148238   
                  Manufacturing                                         27670.428115   
                  Medical/Health Services                              219148.462972   
                  Retail                                                10322.424056   
                  Transportation                                        35988.013518   
                  Wholesale                                             16465.204974   
Goods_Pickup      Agriculture/Mining                                       39.140938   
                  Construction                                           4765.481593   
                  Education/Other public services                        2087.337979   
                  Info/Finance/Insurance/Real Estate/Professional...     7683.308320   
                  Leisure/Accommodations and Food                         137.742969   
                  Manufacturing                                         10195.380932   
                  Medical/Health Services                                6467.488856   
                  Retail                                                 1250.818955   
                  Transportation                                         2844.424115   
                  Wholesale                                              2206.314399   
Home              Agriculture/Mining                                       39.140938   
                  Construction                                           2644.248805   
                  Education/Other public services                        6291.512737   
                  Industrial/Utilities                                    385.343598   
                  Info/Finance/Insurance/Real Estate/Professional...     1797.777390   
                  Leisure/Accommodations and Food                         137.742969   
                  Manufacturing                                           525.449968   
                  Medical/Health Services                                 200.692446   
                  Retail                                                  930.183991   
                  Transportation                                          971.357064   
                  Wholesale                                               383.044463   
Maintenance/Other Agriculture/Mining                                     1218.474388   
                  Cons

In [90]:
i = 'industry_group'  
pvt_ind_act = pd.pivot_table(df_trips, values='company_id', index=i,
                             columns='d_act_seg_name', aggfunc='count').fillna(0) #, sort=True) For sort, default is True. 
pvt_ind_act_w = pd.pivot_table(df_trips, values='expnsn_factor', index=i,
                             columns='d_act_seg_name', aggfunc='sum').fillna(0).round() #, sort=True) For sort, default is True.    
pvt_ind_act
pvt_ind_act_w

d_act_seg_name,Base,Goods_Delivery,Goods_Pickup,Home,Maintenance/Other,Service
industry_group,,,,,,
Agriculture/Mining,1747.0,937.0,39.0,125.0,1696.0,5284.0
Construction,26097.0,15008.0,4765.0,9517.0,49580.0,63845.0
Education/Other public services,52820.0,1732.0,2087.0,14433.0,121087.0,291525.0
Industrial/Utilities,2805.0,727.0,0.0,618.0,3572.0,10159.0
Info/Finance/Insurance/Real Estate/Professional services,33407.0,1237.0,7683.0,11172.0,69878.0,95130.0
Leisure/Accommodations and Food,6707.0,1534.0,138.0,1786.0,7899.0,13541.0
Manufacturing,10781.0,27815.0,10195.0,1872.0,27584.0,28093.0
Medical/Health Services,31772.0,223656.0,6467.0,2865.0,82002.0,38332.0
Retail,13105.0,10322.0,1251.0,3748.0,25794.0,27169.0


In [91]:
pvt_act_plc = pd.pivot_table(df_trips, values='company_id', index='d_act_seg_name',
                             columns='d_plc_seg_name', aggfunc='count').fillna(0) #, sort=True) For sort, default is True.
pvt_act_plc_w = pd.pivot_table(df_trips, values='expnsn_factor', index='d_act_seg_name',
                             columns='d_plc_seg_name', aggfunc='sum').fillna(0).round() #, sort=True) For sort, default is True.
pvt_act_plc
# pvt_act_plc_w

d_plc_seg_name,Gas,"Industrial, Agriculture, or Construction",Office,Other,Residential,Retail and Restaurant,Truck Terminal or Parking,Warehouse
d_act_seg_name,,,,,,,,
Base,2.0,115.0,657.0,22.0,75.0,132.0,11.0,353.0
Goods_Delivery,39.0,105.0,249.0,37.0,1192.0,158.0,4.0,176.0
Goods_Pickup,1.0,81.0,36.0,16.0,12.0,37.0,8.0,46.0
Home,0.0,0.0,1.0,1.0,378.0,1.0,0.0,11.0
Maintenance/Other,491.0,60.0,260.0,817.0,117.0,675.0,29.0,97.0
Service,31.0,382.0,1321.0,342.0,1361.0,358.0,4.0,386.0


In [92]:
out_file = dataset + f'_StopDuration_{current_date}.xlsx'
out_path = os.path.join(project_path, out_data_dir)
if not os.path.exists(out_path):
   os.makedirs(out_path)
out_path = os.path.join(project_path, out_data_dir, out_file)
xl_writer = pd.ExcelWriter(out_path, engine='openpyxl')

In [93]:
sn1 = 'StopDur_byAct&Ind'
sn2 = 'StopDur_byAct&Ind_w'
sn3 = 'Ind-DesAct'
sn4 = 'Ind-DesAct_w'
sn5 = 'DesAct-DesPlc'
sn6 = 'DesAct-DesPlc_w'
AvgStop.to_excel(xl_writer, sheet_name=sn1) # , index_label='index'
AvgStop_w.to_excel(xl_writer, sheet_name=sn2) # , index_label='index'
# pvt_ind_act.to_excel(xl_writer, sheet_name=sn3) # , index_label='index'
# pvt_ind_act_w.to_excel(xl_writer, sheet_name=sn4) # , index_label='index'
# pvt_act_plc.to_excel(xl_writer, sheet_name=sn5) # , index_label='index'
# pvt_act_plc_w.to_excel(xl_writer, sheet_name=sn6) # , index_label='index'
xl_writer.close()

try:
    df_trips.drop(columns='dur_by_weight', inplace=True)
except:
    pass

#### Start Creating the Routes

In [94]:
cols = ['route_id', 'industry_group', 'veh_type',        
        'trip_count', 'g_stops', 's_stops', 'm_stops', 'b_stops', 'h_stops',        
        'primary_purp', 'customer_type',
        'start_tod', 'end_tod', 'route_dur_hr', 'cumlv_dur', 'durations_match',
        'start_activity', 'end_activity',
        'start_plc_seg', 'end_plc_seg',
        'act_seg_seq', 'plc_seg_seq',
        'headquarters', 'hq_taz',
        'tot_distance',
        'company_id', 'vehicle_id', 'driver_id',
        'participation_type', 'expnsn_factor', 'estab_taz', 'trips']
# g_stops = goods stops, s for services, m for maintenance/other, b for base, h for home. (These are super activities.)
# Primary purpose: Goods, Services, Maintenance/Other
# Customer type: Residential, Non-residential, Mixed
# start_tod = starting time of day, it is the first departure; end_tod = ending time of day, it is the last arrival.
# route_dur_hr = Duration of the route based on its start and end times.
# cumlv_dur = Duration of the route based on trip and stay durations.
# durations_match = True if route durations from the two methods match; else False.
# start_activity or end_activity: Starting/ending activity at O/D
# start_plc_seg or end_plc_seg: Starting/ending place (Base, Home, Warehouse/DC (distribution center)/Transport Node, Other)
# headquarters: Primary point of return. Most of the time it is the base.
# hq_taz: TAZ of the primary point of return.

if dataset=='TNC': cols.insert(-1, 'company_name')
    
df_routes = pd.DataFrame(columns = cols)

In [95]:
# Iterate through the trip dataframe.
for i, row in df_trips.iterrows():
    
    # If this trip is the first trip of the route, initialize a route for it.
    if row['trip_number'] == 1:
        df_temp_rt = pd.DataFrame(columns=cols, index=[0])
        
        # Generate a route ID.
        date = str(df_trips.at[i, 'travel_date'])[:10]
        date = date.replace('-', '')
        date = int(date) * 10000
        df_temp_rt['route_id'] = date + row['vehicle_id']
        
        # Copy common attributes from the trip row to the temporary route dataframe.
        for c in df_temp_rt.columns.to_list():
            if c in df_trips.columns.to_list():
                df_temp_rt[c] = row[c]
        
        # Initialize route variables.
        trips = []
        activities = [row['o_act_seg']]
        places = [row['o_plc_seg']]
        
        # Set route variables that are known.
        df_temp_rt['start_activity'] = row['o_act_seg']
        df_temp_rt['start_plc_seg'] = dic_pt2[row['o_plc_seg']]
        df_temp_rt['start_tod'] = row['o_dt']
    
    # Initialize a trip and store the current row values into its attributes.
    t = Trip()    
    t.index = i
    for attr in list(vars(t).keys()):
        if attr == 'index': continue
        #  print(row[str(attr)])
        #  print(getattr(t, attr))
        setattr(t, attr, row[attr])
    
    trips.append(t)
    # Add trip info to the route variables.    
    activities.append(row['d_act_seg'])
    places.append(row['d_plc_seg'])
    
    # Finalize the route if this is the last trip of the route.
    if row['last_trip']:
        df_temp_rt.at[0, 'act_seg_seq'] = activities
        df_temp_rt.at[0, 'plc_seg_seq'] = places
        df_temp_rt.at[0, 'trips'] = trips
                
        # Set route variables that are known.        
        df_temp_rt['end_activity'] = row['d_act_seg']
        df_temp_rt['end_plc_seg'] = dic_pt2[row['d_plc_seg']]
        df_temp_rt['end_tod'] = row['d_at']
        
        # Attach the completed route to the route dataframe.
        df_routes = pd.concat([df_routes, df_temp_rt], axis=0, ignore_index=True)
        
#     if i == 300: break
df_routes.tail()

,route_id,industry_group,veh_type,trip_count,g_stops,s_stops,m_stops,b_stops,h_stops,primary_purp,customer_type,start_tod,end_tod,route_dur_hr,cumlv_dur,durations_match,start_activity,end_activity,start_plc_seg,end_plc_seg,act_seg_seq,plc_seg_seq,headquarters,hq_taz,tot_distance,company_id,vehicle_id,driver_id,participation_type,expnsn_factor,estab_taz,trips
1569,202210111855,Construction,MUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-11 08:04:00,2022-10-11 18:31:00,NaN,NaN,NaN,B,B,Office,Warehouse,"[B, M, P, D, D, B, S, B, M, B]","[2, 6, 7, 2, 2, 2, 1, 3, 1, 3]",,0,NaN,600154,1855,2394,smartphone,247.696367,3366,"[<__main__.Trip object at 0x0000029280B72748>,..."
1570,202210111856,Construction,MUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-11 08:03:00,2022-10-11 11:43:00,NaN,NaN,NaN,M,M,Residential,Truck Terminal or Parking,"[M, M, P, D, M]","[1, 6, 7, 7, 8]",,0,NaN,600154,1856,2395,smartphone,247.696367,3366,"[<__main__.Trip object at 0x0000029280B57B08>,..."
1571,202210111857,Construction,MUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-11 08:03:00,2022-10-11 16:57:00,NaN,NaN,NaN,B,B,Office,Warehouse,"[B, M, P, D, D, D, B, S, S, B]","[2, 6, 7, 2, 5, 5, 2, 3, 3, 3]",,0,NaN,600154,1857,2396,smartphone,247.696367,3366,"[<__main__.Trip object at 0x0000029280B68B48>,..."
1572,202210111858,Construction,MUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-11 08:03:00,2022-10-11 11:45:00,NaN,NaN,NaN,B,D,Office,Truck Terminal or Parking,"[B, M, P, S, D]","[2, 6, 7, 2, 8]",,0,NaN,600154,1858,2397,smartphone,247.696367,3366,"[<__main__.Trip object at 0x0000029280B68788>,..."
1573,202210111859,Construction,MUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-11 08:03:00,2022-10-11 12:03:00,NaN,NaN,NaN,B,B,Office,Truck Terminal or Parking,"[B, M, P, D, S, D, B]","[2, 6, 7, 7, 4, 7, 8]",,0,NaN,600154,1859,2398,smartphone,247.696367,3366,"[<__main__.Trip object at 0x0000029280B68C88>,..."


In [96]:
for i, row in df_routes.iterrows():
    # Identify number of stops by type of stop.        
    df_routes.at[i, 'trip_count'] = len(row['trips'])
    counts = collections.Counter(row['act_seg_seq'][1:])
    df_routes.at[i, 'g_stops'] = counts['P'] + counts['D']
    df_routes.at[i, 's_stops'] = counts['S']
    df_routes.at[i, 'm_stops'] = counts['M']
    df_routes.at[i, 'b_stops'] = counts['B']
    df_routes.at[i, 'h_stops'] = counts['H']
    
    # Identify the primary purpose of the route.
    if 'P' in row['act_seg_seq'][1:]:
        purp = 'Goods'
    elif 'D' in row['act_seg_seq'][1:]:
        purp = 'Goods'        
    elif 'S' in row['act_seg_seq'][1:]:
        purp = 'Service'
    else:
        purp = 'Maintenance/Other'
    df_routes.at[i, 'primary_purp'] = purp    
    
    # Find total distance traveled on the route.
    dist = 0
    for t in row['trips']:
        dist += t.trip_dist
    df_routes.at[i, 'tot_distance'] = dist
    
df_routes.head()

,route_id,industry_group,veh_type,trip_count,g_stops,s_stops,m_stops,b_stops,h_stops,primary_purp,customer_type,start_tod,end_tod,route_dur_hr,cumlv_dur,durations_match,start_activity,end_activity,start_plc_seg,end_plc_seg,act_seg_seq,plc_seg_seq,headquarters,hq_taz,tot_distance,company_id,vehicle_id,driver_id,participation_type,expnsn_factor,estab_taz,trips
0,202209302062,Education/Other public services,LCV,4,0,1,1,2,0,Service,NaN,2022-09-30 10:00:00,2022-09-30 14:20:00,NaN,NaN,NaN,B,B,Residential,Residential,"[B, M, B, S, B]","[1, 5, 1, 4, 1]",,0,18.5,100012,2062,2589,online,177.825038,2284,"[<__main__.Trip object at 0x000002934A83CA88>,..."
1,202209302063,Education/Other public services,LCV,3,0,1,1,1,0,Service,NaN,2022-09-30 14:00:00,2022-09-30 17:22:00,NaN,NaN,NaN,B,B,Residential,Residential,"[B, S, M, B]","[1, 4, 6, 1]",,0,16.9,100012,2063,2590,online,177.825038,2284,"[<__main__.Trip object at 0x000002930D7AB088>,..."
2,202209302064,Education/Other public services,LCV,5,0,2,1,2,0,Service,NaN,2022-09-30 10:00:00,2022-09-30 15:03:00,NaN,NaN,NaN,B,B,Residential,Residential,"[B, S, B, S, M, B]","[1, 2, 1, 2, 6, 1]",,0,90,100012,2064,2591,online,177.825038,2284,"[<__main__.Trip object at 0x000002930D7B87C8>,..."
3,202209161405,Agriculture/Mining,LCV,2,0,0,2,0,0,Maintenance/Other,NaN,2022-09-16 09:07:36,2022-09-16 09:42:00,NaN,NaN,NaN,H,M,Residential,Office,"[H, M, M]","[1, 5, 2]",,0,9.1,100021,1405,1974,smartphone,14.882969,89,"[<__main__.Trip object at 0x00000292855E5448>,..."
4,202209161406,Agriculture/Mining,LCV,4,0,2,2,0,0,Service,NaN,2022-09-16 08:55:00,2022-09-16 09:41:00,NaN,NaN,NaN,B,S,Office,Other,"[B, S, M, M, S]","[2, 1, 2, 6, 4]",,0,19.2,100021,1406,1975,smartphone,14.882969,89,"[<__main__.Trip object at 0x0000029281E9D848>,..."


In [97]:
for i, t in enumerate(df_routes.at[0, 'trips']):    
    print(f'\nTrip #{i+1}:')
    for e in zip(list(vars(t).keys()), list(vars(t).values())):
        print(e)    


Trip #1:
('index', 0)
('o_act', 1.0)
('o_place_type', 11.0)
('o_place_name', 'BASE')
('o_address', '2801 B ST # 55')
('o_lon', -117.133702)
('o_lat', 32.717871)
('o_taz', 2284)
('d_act', 11)
('d_place_type', 6)
('d_place_name', 'PET SMART')
('d_address', '1660 Camino De La Reina d')
('d_lon', -117.149017)
('d_lat', 32.770912)
('d_taz', 2091)
('trip_dist', 5.6)
('o_dt', Timestamp('2022-09-30 10:00:00'))
('d_at', Timestamp('2022-09-30 10:20:00'))
('nt_dt', Timestamp('2022-09-30 11:10:00'))
('travel_time', 20)
('stop_duration', 50)
('cargo_pickup', nan)
('cargo_delivery', nan)
('travel_date', Timestamp('2022-09-30 00:00:00'))
('last_trip', False)

Trip #2:
('index', 1)
('o_act', 11.0)
('o_place_type', 6.0)
('o_place_name', 'PET SMART')
('o_address', '1660 Camino De La Reina d')
('o_lon', -117.149017)
('o_lat', 32.770912)
('o_taz', 2091)
('d_act', 1)
('d_place_type', 11)
('d_place_name', 'BASE')
('d_address', '2801 B ST # 55')
('d_lon', -117.133702)
('d_lat', 32.717871)
('d_taz', 2284)
('

In [98]:
# Determine customer type.

# Customer Types at Stops (only applies to routes with Goods and/or Service purposes):
# a. Residential Only (households, including multi-family buildings)
# b. Non-residential Only (commercial, public/government)
# c. Mixed Residential and Non-residential
dic_cstmr_typ = {
    'ro' : 'Residential Only',
    'nro': 'Non-Residential Only',
    'm'  : 'Mixed Residential and Non-residential',
    'nc' : 'No Customer'
}

# plc_seg_code: plc_seg_name
# 1: Residential
# 2: Office
# 3: Warehouse
# 4: Other   A closer look at the trip table indicated most of these places are non-residential.
# 5: Retail and Restaurant
# 6: Gas
# 7: Industrial, Agriculture, or Construction
# 8: Truck Terminal or Parking

lst_res = [1]
lst_non_res = [2, 3, 4, 5, 6, 7, 8]

# Logic: if no relevant place is visited: No Customer;
#        else if every visited place is res: Residential Only;
#        else if there's at least one residential place that has been visited: Mixed;
#        else: Non-Residential Only.

def identify_customer(lrvp):  # lrvp is the list of relevant, visited places.    
    if not lrvp: return 'nc'  # if input list is empty, no customer has been served, or no
                              # good has been delivered or picked up.
    result = 'ro'
    for plc in lrvp:
        if plc not in lst_res:
            result = ''
    if result == 'ro': return result
    result = 'm'
    for plc in lrvp:
        if plc not in lst_non_res:
            return result
    result = 'nro'
    return result
    

for i, route in df_routes.iterrows():
    acts = route['act_seg_seq']
    plcs = route['plc_seg_seq']
    plcs_cleaned = []
    for j, act in enumerate(acts):
        if act in ['S', 'P', 'D']: plcs_cleaned.append(plcs[j])
    cstmr = identify_customer(plcs_cleaned)
    df_routes.at[i, 'customer_type'] = dic_cstmr_typ[cstmr]

In [99]:
df_routes[['act_seg_seq', 'plc_seg_seq', 'customer_type']]

,act_seg_seq,plc_seg_seq,customer_type
0,"[B, M, B, S, B]","[1, 5, 1, 4, 1]",Non-Residential Only
1,"[B, S, M, B]","[1, 4, 6, 1]",Non-Residential Only
2,"[B, S, B, S, M, B]","[1, 2, 1, 2, 6, 1]",Non-Residential Only
3,"[H, M, M]","[1, 5, 2]",No Customer
4,"[B, S, M, M, S]","[2, 1, 2, 6, 4]",Mixed Residential and Non-residential
5,"[S, H, M, S]","[1, 1, 5, 6]",Mixed Residential and Non-residential
6,"[B, S, M, S, S]","[2, 1, 6, 6, 2]",Mixed Residential and Non-residential
7,"[B, S, H, M, M]","[2, 1, 1, 6, 2]",Residential Only
8,"[B, S, M, H]","[2, 1, 6, 1]",Residential Only
9,"[B, D, M]","[2, 6, 7]",Non-Residential Only


In [100]:
# Check how many of the routes go outside of the SANDAG TAZs.
a = len(df_routes)
b = len(df_routes.loc[pd.isnull(df_routes['tot_distance'])])
print(f'{b} routes out of {a:,} routes have trips that either start or end in an external TAZ.')
print("'tot_distance' field of these routes has been marked as 'NA'.")

114 routes out of 1,574 routes have trips that either start or end in an external TAZ.
'tot_distance' field of these routes has been marked as 'NA'.


In [101]:
# Determine what is the point of return of the route. It's either base or home.
for i, row in df_routes.iterrows():
    counts_act = collections.Counter(row['act_seg_seq'])
#     counts_plc = collections.Counter(row['plc_seg_seq'])
    if counts_act['B'] >= counts_act['H'] and counts_act['B'] > 0:
        df_routes.at[i, 'headquarters'] = 'B'
    elif counts_act['H'] > counts_act['B']:
        df_routes.at[i, 'headquarters'] = 'H'
#     elif counts_plc[3] >= counts_plc[8] and counts_plc[3] > 0:
#         df_routes.at[i, 'headquarters'] = 'Warehouse'
#     elif counts_plc[8] > counts_act[3]:
#         df_routes.at[i, 'headquarters'] = 'Truck Terminal'
    else:
        df_routes.at[i, 'headquarters'] = 'Unknown'
df_routes.groupby('headquarters').size().reset_index(name='count')

,headquarters,count
0,B,1362
1,H,137
2,Unknown,75


In [102]:
df_routes.loc[df_routes['headquarters']=='Unknown'].head()

,route_id,industry_group,veh_type,trip_count,g_stops,s_stops,m_stops,b_stops,h_stops,primary_purp,customer_type,start_tod,end_tod,route_dur_hr,cumlv_dur,durations_match,start_activity,end_activity,start_plc_seg,end_plc_seg,act_seg_seq,plc_seg_seq,headquarters,hq_taz,tot_distance,company_id,vehicle_id,driver_id,participation_type,expnsn_factor,estab_taz,trips
45,202208110767,Agriculture/Mining,LCV,8,0,6,2,0,0,Service,Mixed Residential and Non-residential,2022-08-11 10:26:00,2022-08-11 12:44:00,NaN,NaN,NaN,S,M,"Industrial, Agriculture, or Construction",Gas,"[S, M, S, S, S, S, S, S, M]","[7, 6, 1, 2, 2, 1, 1, 1, 6]",Unknown,0,60.3,100298,767,961,smartphone,14.882969,953,"[<__main__.Trip object at 0x0000029281E982C8>,..."
117,202209191616,Manufacturing,LCV,7,3,1,3,0,0,Goods,Mixed Residential and Non-residential,2022-09-19 15:00:00,2022-09-19 18:45:00,NaN,NaN,NaN,S,M,Residential,Office,"[S, M, S, P, D, D, M, M]","[1, 1, 2, 7, 2, 5, 5, 2]",Unknown,0,18.3,102839,1616,2018,online,46.608727,267,"[<__main__.Trip object at 0x0000029280633E88>,..."
194,202208110821,Manufacturing,MUT,11,2,1,8,0,0,Goods,Non-Residential Only,2022-08-11 05:39:00,2022-08-11 14:44:00,NaN,NaN,NaN,M,M,Other,"Industrial, Agriculture, or Construction","[M, S, M, M, M, M, M, M, M, P, P, M]","[4, 2, 4, 4, 7, 5, 5, 7, 4, 7, 7, 7]",Unknown,0,125.1,103903,821,1056,smartphone,356.546165,3895,"[<__main__.Trip object at 0x0000029280790088>,..."
195,202208110822,Manufacturing,MUT,5,3,0,2,0,0,Goods,Non-Residential Only,2022-08-11 03:52:00,2022-08-11 14:51:00,NaN,NaN,NaN,S,M,Office,"Industrial, Agriculture, or Construction","[S, P, D, M, D, M]","[2, 7, 4, 7, 4, 7]",Unknown,0,NaN,103903,822,1057,smartphone,356.546165,3895,"[<__main__.Trip object at 0x0000029280784CC8>,..."
200,202208110827,Manufacturing,SUT,9,4,0,5,0,0,Goods,Mixed Residential and Non-residential,2022-08-11 03:46:00,2022-08-11 13:25:00,NaN,NaN,NaN,S,M,Office,"Industrial, Agriculture, or Construction","[S, P, M, M, D, M, M, D, D, M]","[2, 7, 7, 4, 7, 7, 6, 1, 1, 7]",Unknown,0,193.1,103903,827,1062,smartphone,144.077505,3895,"[<__main__.Trip object at 0x00000292807BB208>,..."


In [103]:
def find_hq_taz(hq, lst_act, lst_trips):
    if hq in 'BH':
        pos = lst_act.index(hq)
        if pos == 0:
            return lst_trips[0].o_taz
        else:            
            return lst_trips[pos-1].d_taz
    else:
        return -1
    
df_routes['hq_taz'] = df_routes.apply(lambda x: find_hq_taz(x.headquarters, x.act_seg_seq, x.trips), axis=1)
df_routes.head()

,route_id,industry_group,veh_type,trip_count,g_stops,s_stops,m_stops,b_stops,h_stops,primary_purp,customer_type,start_tod,end_tod,route_dur_hr,cumlv_dur,durations_match,start_activity,end_activity,start_plc_seg,end_plc_seg,act_seg_seq,plc_seg_seq,headquarters,hq_taz,tot_distance,company_id,vehicle_id,driver_id,participation_type,expnsn_factor,estab_taz,trips
0,202209302062,Education/Other public services,LCV,4,0,1,1,2,0,Service,Non-Residential Only,2022-09-30 10:00:00,2022-09-30 14:20:00,NaN,NaN,NaN,B,B,Residential,Residential,"[B, M, B, S, B]","[1, 5, 1, 4, 1]",B,2284,18.5,100012,2062,2589,online,177.825038,2284,"[<__main__.Trip object at 0x000002934A83CA88>,..."
1,202209302063,Education/Other public services,LCV,3,0,1,1,1,0,Service,Non-Residential Only,2022-09-30 14:00:00,2022-09-30 17:22:00,NaN,NaN,NaN,B,B,Residential,Residential,"[B, S, M, B]","[1, 4, 6, 1]",B,2284,16.9,100012,2063,2590,online,177.825038,2284,"[<__main__.Trip object at 0x000002930D7AB088>,..."
2,202209302064,Education/Other public services,LCV,5,0,2,1,2,0,Service,Non-Residential Only,2022-09-30 10:00:00,2022-09-30 15:03:00,NaN,NaN,NaN,B,B,Residential,Residential,"[B, S, B, S, M, B]","[1, 2, 1, 2, 6, 1]",B,2284,90,100012,2064,2591,online,177.825038,2284,"[<__main__.Trip object at 0x000002930D7B87C8>,..."
3,202209161405,Agriculture/Mining,LCV,2,0,0,2,0,0,Maintenance/Other,No Customer,2022-09-16 09:07:36,2022-09-16 09:42:00,NaN,NaN,NaN,H,M,Residential,Office,"[H, M, M]","[1, 5, 2]",H,147,9.1,100021,1405,1974,smartphone,14.882969,89,"[<__main__.Trip object at 0x00000292855E5448>,..."
4,202209161406,Agriculture/Mining,LCV,4,0,2,2,0,0,Service,Mixed Residential and Non-residential,2022-09-16 08:55:00,2022-09-16 09:41:00,NaN,NaN,NaN,B,S,Office,Other,"[B, S, M, M, S]","[2, 1, 2, 6, 4]",B,89,19.2,100021,1406,1975,smartphone,14.882969,89,"[<__main__.Trip object at 0x0000029281E9D848>,..."


##### Now that headquarters of routes are known, go back to df_trips and update distance to headquarters for each trip.

In [104]:
# Create a dictionary that gets route_id and returns hq_taz.
dic_rid_hq_taz = dict(zip(df_routes.route_id, df_routes.hq_taz))

# Specify hq_taz of trips in the trips dataframe.
def omit_dashes(s):
    return s.replace('-', '')

df_trips['route_id'] = df_trips['travel_date'].astype(str).str[:10].apply(omit_dashes).astype('int64') * 10000
df_trips['route_id'] += df_trips.vehicle_id
df_trips.hq_taz = df_trips.route_id.map(dic_rid_hq_taz)

df_trips[['travel_date', 'vehicle_id', 'route_id', 'hq_taz']].head()

,travel_date,vehicle_id,route_id,hq_taz
0,2022-09-30,2062,202209302062,2284
1,2022-09-30,2062,202209302062,2284
2,2022-09-30,2062,202209302062,2284
3,2022-09-30,2062,202209302062,2284
4,2022-09-30,2063,202209302063,2284


In [105]:
# Add route primary purposes to trips
dic_rte_purp = dict(zip(df_routes.route_id, df_routes.primary_purp))
df_trips['route_purpose'] = df_trips.route_id.map(dic_rte_purp)

In [106]:
# Add route primary customer type to trips
dic_rte_cust = dict(zip(df_routes.route_id, df_routes.customer_type)) 
df_trips['route_customers'] = df_trips.route_id.map(dic_rte_cust)

In [107]:
# Update distance to headquarters.
df_trips['orgn_to_hq_dist'] = df_trips.apply(lambda x: find_dist(x['o_taz'], x['hq_taz'],
                                                           dic_v[x['veh_type']],
                                                           dic_tod[x['tod']]), axis=1)

In [108]:
df_trips.head()

,company_id,industry_group,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz,industry_code,o_dt,d_at,nt_dt,last_trip,tod,toll_in_cents_from_base,gen_tt_from_base,route_id,route_purpose,route_customers
0,100012,Education/Other public services,2062,2589,1,B,Base,1,Residential,M,Maintenance/Other,5,Retail and Restaurant,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,5.6,,2284,0.1,20,50,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 10:00:00,2022-09-30 10:20:00,2022-09-30 11:10:00,False,MD,0.0,15.26,202209302062,Service,Non-Residential Only
1,100012,Education/Other public services,2062,2589,2,M,Maintenance/Other,5,Retail and Restaurant,B,Base,1,Residential,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,6.9,,2284,6.9,12,18,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 11:10:00,2022-09-30 11:22:00,2022-09-30 11:40:00,False,MD,0.0,0.55,202209302062,Service,Non-Residential Only
2,100012,Education/Other public services,2062,2589,3,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,2.8,,2284,0.1,20,120,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 11:40:00,2022-09-30 12:00:00,2022-09-30 14:00:00,False,MD,0.0,8.77,202209302062,Service,Non-Residential Only
3,100012,Education/Other public services,2062,2589,4,S,Service,4,Other,B,Base,1,Residential,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,3.2,,2284,3.2,20,NaN,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 14:00:00,2022-09-30 14:20:00,NaT,True,MD,0.0,0.55,202209302062,Service,Non-Residential Only
4,100012,Education/Other public services,2063,2590,1,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,Soto St,-117.237975,32.752893,828,8.1,,2284,0.1,11,154,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 14:00:00,2022-09-30 14:11:00,2022-09-30 16:45:00,False,MD,0.0,11.99,202209302063,Service,Non-Residential Only


In [109]:
print(df_trips.groupby(['route_purpose']).size().to_string(), "\n")
print(df_trips.groupby(['route_customers']).size().to_string())

route_purpose
Goods                3366
Maintenance/Other     455
Service              6866 

route_customers
Mixed Residential and Non-residential    4858
No Customer                               412
Non-Residential Only                     4145
Residential Only                         1272


####  Run a Few Checks for Quality Control of the Route Dataframe

In [110]:
# Find the route start to end duration in minutes.
df_routes['route_dur_min'] = ((df_routes['end_tod'] - df_routes['start_tod'])
                             .dt.total_seconds()/60).round()

# Find the route duration in minutes based on its trips and stays durations.
for i, route in df_routes.iterrows():
    duration = 0
    trips = route['trips']
    for j, t in enumerate(trips):
        if j != len(trips)-1:
            duration += (t.travel_time + t.stop_duration)
        else:
            duration += t.travel_time
    duration = round(duration)
    df_routes.at[i, 'cumlv_dur'] = duration
    
    # Specify if route durations from the two methods are off by more than tolerance minutes.
    tol = 15
    if abs(duration - route['route_dur_min']) >= tol:
        df_routes.at[i, 'durations_match'] = False
    else:
        df_routes.at[i, 'durations_match'] = True

misses = len(df_routes.loc[df_routes['durations_match'] == False])
matches = len(df_routes.loc[df_routes['durations_match'] != False])
print(f"Number of routes with duration matches = {matches}; misses = {misses} (tolerance = {tol} minutes)")

Number of routes with duration matches = 1572; misses = 2 (tolerance = 15 minutes)


In [111]:
df_routes.loc[df_routes['durations_match'] == False].head()

,route_id,industry_group,veh_type,trip_count,g_stops,s_stops,m_stops,b_stops,h_stops,primary_purp,customer_type,start_tod,end_tod,route_dur_hr,cumlv_dur,durations_match,start_activity,end_activity,start_plc_seg,end_plc_seg,act_seg_seq,plc_seg_seq,headquarters,hq_taz,tot_distance,company_id,vehicle_id,driver_id,participation_type,expnsn_factor,estab_taz,trips,route_dur_min
116,202209151550,Manufacturing,LCV,31,0,22,8,0,1,Service,Mixed Residential and Non-residential,2022-09-15 08:05:16,2022-09-15 17:51:49,NaN,571,False,S,H,Residential,Retail and Restaurant,"[S, M, S, S, S, S, S, S, M, S, S, M, S, M, M, ...","[1, 6, 2, 1, 5, 3, 3, 1, 5, 2, 3, 5, 1, 6, 6, ...",H,190,264.9,102839,1550,1987,smartphone,46.608727,267,"[<__main__.Trip object at 0x0000029280628308>,...",587.0
958,202209201598,Construction,MUT,20,0,16,4,0,0,Service,Mixed Residential and Non-residential,2022-09-20 08:10:30,2022-09-20 14:05:19,NaN,340,False,M,S,Gas,Retail and Restaurant,"[M, S, S, S, S, S, S, M, S, S, M, S, M, S, S, ...","[6, 5, 5, 4, 5, 5, 5, 3, 5, 5, 3, 1, 3, 2, 2, ...",Unknown,-1,140.8,127819,1598,2042,smartphone,247.696367,2848,"[<__main__.Trip object at 0x0000029280C9D108>,...",355.0


In [112]:
# Determine if the route has extended beyond the first day. This does not require that the duration of a route is more
# than 24 hours, rather if the first departure is in one calendar day while the last arrival is in another day, the
# route will be flagged as Multi-Day.
try:
    df_routes.insert(13, 'multiday_route', False)
except:
    pass
df_routes['multiday_route'] = df_routes['start_tod'].dt.date < df_routes['end_tod'].dt.date    

In [113]:
df_routes.loc[df_routes['multiday_route']==True].head()

,route_id,industry_group,veh_type,trip_count,g_stops,s_stops,m_stops,b_stops,h_stops,primary_purp,customer_type,start_tod,end_tod,multiday_route,route_dur_hr,cumlv_dur,durations_match,start_activity,end_activity,start_plc_seg,end_plc_seg,act_seg_seq,plc_seg_seq,headquarters,hq_taz,tot_distance,company_id,vehicle_id,driver_id,participation_type,expnsn_factor,estab_taz,trips,route_dur_min
370,202209221571,Retail,SUT,22,0,11,9,2,0,Service,Mixed Residential and Non-residential,2022-09-22 08:03:10,2022-09-23 00:15:00,True,NaN,971,True,H,M,Residential,Office,"[H, M, M, M, B, S, B, S, M, M, S, S, S, S, S, ...","[1, 4, 4, 4, 2, 2, 2, 3, 4, 4, 3, 3, 1, 5, 2, ...",B,3870,85,116019,1571,2115,online,364.202354,3870,"[<__main__.Trip object at 0x000002928117C0C8>,...",972.0
427,202206280030,Wholesale,LCV,3,0,1,1,1,0,Service,Residential Only,2022-06-28 07:18:07,2022-06-29 00:00:59,True,NaN,1001,True,B,M,Residential,Residential,"[B, S, B, M]","[1, 1, 1, 1]",B,3262,8.1,117585,30,51,smartphone,50.460422,3262,"[<__main__.Trip object at 0x0000029281140788>,...",1003.0
525,202207200117,Wholesale,LCV,5,2,1,0,2,0,Goods,Non-Residential Only,2022-07-20 10:32:41,2022-07-21 00:23:59,True,NaN,830,True,B,B,Office,Office,"[B, D, B, P, S, B]","[2, 4, 2, 7, 4, 2]",B,2255,114.3,119620,117,207,smartphone,50.460422,2255,"[<__main__.Trip object at 0x000002928112BF88>,...",831.0
862,202208251242,Construction,LCV,4,0,2,1,0,1,Service,Non-Residential Only,2022-08-25 16:43:00,2022-08-26 00:17:59,True,NaN,455,True,B,H,Office,Residential,"[B, S, M, S, H]","[2, 2, 6, 2, 1]",B,2344,13.1,126973,1242,1644,online,116.750222,2344,"[<__main__.Trip object at 0x0000029280F1F308>,...",455.0
899,202208090664,Transportation,LCV,2,0,2,0,0,0,Service,Non-Residential Only,2022-08-09 08:41:37,2022-08-10 09:19:07,True,NaN,1476,True,M,S,Office,Office,"[M, S, S]","[2, 2, 2]",Unknown,-1,23.6,127188,664,978,smartphone,63.910364,1974,"[<__main__.Trip object at 0x0000029280D9CA88>,...",1478.0


In [114]:
# Add a column that indicates if the route has any warning.
df_routes['has_warning'] = 0

In [115]:
# Identify routes that vehicle goes from Base to Base, or it goes from Home to Home.
for i, route in df_routes.iterrows():
    flag = False
    acts = route['act_seg_seq']
    act = acts[0]
    for next_act in acts[1:]:
        if (act=='H' and next_act=='H') or (act=='B' and next_act=='B'):
            flag = True
            break
        act = next_act
    df_routes.at[i, 'warn_BB_or_HH'] = flag*1
df_routes['warn_BB_or_HH'] = df_routes['warn_BB_or_HH'].astype('int8')

In [116]:
# Add warning columns.    
    
# Activity at the route origin is to deliver/pickup goods:
df_routes['warn_o_act_G'] = df_routes['act_seg_seq'].apply(lambda l: 1 if l[0]=='G' else 0)

# Activity at the route origin is to provide services:
df_routes['warn_o_act_S'] = df_routes['act_seg_seq'].apply(lambda l: 1 if l[0]=='S' else 0)

# Activity at the route destination is either to provide service or deliver/pickup goods:
df_routes['warn_d_act_SG'] = df_routes['act_seg_seq'].apply(lambda l: 1 if l[-1] in ['G', 'S'] else 0)

# Route trips extend beyond 12AM of the first day:
df_routes['warn_next_day'] = df_routes['multiday_route'].astype(int)

# Route duration time calculated from start and end of the tour doesn't match with the one that is calculated by
# summing trip travel times and stop durations:
df_routes['warn_duration'] = 1 - df_routes['durations_match'].astype(int)

# Neither Base nor Home appears in the route activities, including the route origin:
df_routes['warn_no_BH_stops'] = df_routes['act_seg_seq'].apply(lambda l: 0 if ('B' in l or 'H' in l) else 1)

# Neither Goods nor Services appears in the route activities, including the route origin:
df_routes['warn_no_GS_stops'] = df_routes['act_seg_seq'].apply(lambda l: 0 if ('G' in l or 'S' in l) else 1)

# None of Goods, Services, or Maintenance appears in the route activities, including the route origin:
df_routes['warn_no_GSM_stops'] = df_routes['act_seg_seq'].apply(lambda l: 0 if ('G' in l or 'S' in l or 'M' in l) else 1)
df_routes.head()

,route_id,industry_group,veh_type,trip_count,g_stops,s_stops,m_stops,b_stops,h_stops,primary_purp,customer_type,start_tod,end_tod,multiday_route,route_dur_hr,cumlv_dur,durations_match,start_activity,end_activity,start_plc_seg,end_plc_seg,act_seg_seq,plc_seg_seq,headquarters,hq_taz,tot_distance,company_id,vehicle_id,driver_id,participation_type,expnsn_factor,estab_taz,trips,route_dur_min,has_warning,warn_BB_or_HH,warn_o_act_G,warn_o_act_S,warn_d_act_SG,warn_next_day,warn_duration,warn_no_BH_stops,warn_no_GS_stops,warn_no_GSM_stops
0,202209302062,Education/Other public services,LCV,4,0,1,1,2,0,Service,Non-Residential Only,2022-09-30 10:00:00,2022-09-30 14:20:00,False,NaN,260,True,B,B,Residential,Residential,"[B, M, B, S, B]","[1, 5, 1, 4, 1]",B,2284,18.5,100012,2062,2589,online,177.825038,2284,"[<__main__.Trip object at 0x000002934A83CA88>,...",260.0,0,0,0,0,0,0,0,0,0,0
1,202209302063,Education/Other public services,LCV,3,0,1,1,1,0,Service,Non-Residential Only,2022-09-30 14:00:00,2022-09-30 17:22:00,False,NaN,202,True,B,B,Residential,Residential,"[B, S, M, B]","[1, 4, 6, 1]",B,2284,16.9,100012,2063,2590,online,177.825038,2284,"[<__main__.Trip object at 0x000002930D7AB088>,...",202.0,0,0,0,0,0,0,0,0,0,0
2,202209302064,Education/Other public services,LCV,5,0,2,1,2,0,Service,Non-Residential Only,2022-09-30 10:00:00,2022-09-30 15:03:00,False,NaN,303,True,B,B,Residential,Residential,"[B, S, B, S, M, B]","[1, 2, 1, 2, 6, 1]",B,2284,90,100012,2064,2591,online,177.825038,2284,"[<__main__.Trip object at 0x000002930D7B87C8>,...",303.0,0,0,0,0,0,0,0,0,0,0
3,202209161405,Agriculture/Mining,LCV,2,0,0,2,0,0,Maintenance/Other,No Customer,2022-09-16 09:07:36,2022-09-16 09:42:00,False,NaN,33,True,H,M,Residential,Office,"[H, M, M]","[1, 5, 2]",H,147,9.1,100021,1405,1974,smartphone,14.882969,89,"[<__main__.Trip object at 0x00000292855E5448>,...",34.0,0,0,0,0,0,0,0,0,1,0
4,202209161406,Agriculture/Mining,LCV,4,0,2,2,0,0,Service,Mixed Residential and Non-residential,2022-09-16 08:55:00,2022-09-16 09:41:00,False,NaN,46,True,B,S,Office,Other,"[B, S, M, M, S]","[2, 1, 2, 6, 4]",B,89,19.2,100021,1406,1975,smartphone,14.882969,89,"[<__main__.Trip object at 0x0000029281E9D848>,...",46.0,0,0,0,0,1,0,0,0,0,0


In [117]:
# Fill the has_warning column.
warn_cols = [c for c in df_routes.columns if 'warn' in c]
df_routes['has_warning'] = df_routes[warn_cols].max(axis=1)
# df_routes.head(20)

#### Explore the Route Dataframe

##### OPTIONAL: Create a cumulative distribution function for route durations.

##### OPTIONAL: Create a graph for starting/ending activity combinations.
Note: Starting activity is the activity at the origin of the first trip, not at its destination.

#### Report Out the Trip Dataframe

In [118]:
print(f'There are {len(df_trips):,} trips in the dataframe.')

There are 10,687 trips in the dataframe.


In [119]:
df_trips.head()

,company_id,industry_group,vehicle_id,driver_id,trip_number,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_place_name,o_address,o_lon,o_lat,o_taz,d_act,d_place_type,d_place_name,d_address,d_lon,d_lat,d_taz,trip_dist,headquarters,hq_taz,orgn_to_hq_dist,travel_time,stop_duration,veh_type,cargo_pickup,cargo_delivery,travel_date,participation_type,expnsn_factor,estab_taz,industry_code,o_dt,d_at,nt_dt,last_trip,tod,toll_in_cents_from_base,gen_tt_from_base,route_id,route_purpose,route_customers
0,100012,Education/Other public services,2062,2589,1,B,Base,1,Residential,M,Maintenance/Other,5,Retail and Restaurant,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,11,6,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,5.6,,2284,0.1,20,50,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 10:00:00,2022-09-30 10:20:00,2022-09-30 11:10:00,False,MD,0.0,15.26,202209302062,Service,Non-Residential Only
1,100012,Education/Other public services,2062,2589,2,M,Maintenance/Other,5,Retail and Restaurant,B,Base,1,Residential,11.0,6.0,PET SMART,1660 Camino De La Reina d,-117.149017,32.770912,2091,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,6.9,,2284,6.9,12,18,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 11:10:00,2022-09-30 11:22:00,2022-09-30 11:40:00,False,MD,0.0,0.55,202209302062,Service,Non-Residential Only
2,100012,Education/Other public services,2062,2589,3,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,2.8,,2284,0.1,20,120,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 11:40:00,2022-09-30 12:00:00,2022-09-30 14:00:00,False,MD,0.0,8.77,202209302062,Service,Non-Residential Only
3,100012,Education/Other public services,2062,2589,4,S,Service,4,Other,B,Base,1,Residential,10.0,96.0,DOG PARK,2500 Balboa Dr,-117.158124,32.731546,1933,1,11,BASE,2801 B ST # 55,-117.133702,32.717871,2284,3.2,,2284,3.2,20,NaN,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 14:00:00,2022-09-30 14:20:00,NaT,True,MD,0.0,0.55,202209302062,Service,Non-Residential Only
4,100012,Education/Other public services,2063,2590,1,B,Base,1,Residential,S,Service,4,Other,1.0,11.0,BASE,2801 B ST # 55,-117.133702,32.717871,2284,10,96,DOG PARK,Soto St,-117.237975,32.752893,828,8.1,,2284,0.1,11,154,LCV,NaN,NaN,2022-09-30,online,177.825038,2284,9,2022-09-30 14:00:00,2022-09-30 14:11:00,2022-09-30 16:45:00,False,MD,0.0,11.99,202209302063,Service,Non-Residential Only


In [120]:
# Rearrange to have time-related fields next to each other.
cols = ['industry_group', 'trip_number', 'last_trip',
        'travel_date', 'o_dt', 'tod', 'd_at', 'nt_dt', 'travel_time', 'stop_duration', 
        'o_act_seg', 'o_act_seg_name', 'o_plc_seg', 'o_plc_seg_name',
        'd_act_seg', 'd_act_seg_name', 'd_plc_seg', 'd_plc_seg_name',                
        'o_act', 'o_place_type', 'o_taz',
        'd_act', 'd_place_type', 'd_taz',
        'trip_dist',
        'veh_type', 
        'route_id', 'company_id', 'estab_taz', 'vehicle_id', 'driver_id',
        'expnsn_factor', 'participation_type', 'route_purpose', 'route_customers']

if dataset=='TNC':    
    cols.insert(-5, 'company_name')

df_trips = df_trips[cols]
df_trips.head(1)

,industry_group,trip_number,last_trip,travel_date,o_dt,tod,d_at,nt_dt,travel_time,stop_duration,o_act_seg,o_act_seg_name,o_plc_seg,o_plc_seg_name,d_act_seg,d_act_seg_name,d_plc_seg,d_plc_seg_name,o_act,o_place_type,o_taz,d_act,d_place_type,d_taz,trip_dist,veh_type,route_id,company_id,estab_taz,vehicle_id,driver_id,expnsn_factor,participation_type,route_purpose,route_customers
0,Education/Other public services,1,False,2022-09-30,2022-09-30 10:00:00,MD,2022-09-30 10:20:00,2022-09-30 11:10:00,20,50,B,Base,1,Residential,M,Maintenance/Other,5,Retail and Restaurant,1.0,11.0,2284,11,6,2091,5.6,LCV,202209302062,100012,2284,2062,2589,177.825038,online,Service,Non-Residential Only


In [121]:
out_file = dataset + f'_Trips_{current_date}.xlsx'
out_path = os.path.join(project_path, out_data_dir, out_file)
df_trips.to_excel(out_path, sheet_name='Trip Dataframe', index_label='index')

#### Report Out the Route Dataframe

In [122]:
drop_cols = ['multiday_route', 'cumlv_dur', 'durations_match', 'trips', 'headquarters', 'hq_taz']
df_routes.drop(columns=drop_cols, inplace=True)
df_routes.head(2)

,route_id,industry_group,veh_type,trip_count,g_stops,s_stops,m_stops,b_stops,h_stops,primary_purp,customer_type,start_tod,end_tod,route_dur_hr,start_activity,end_activity,start_plc_seg,end_plc_seg,act_seg_seq,plc_seg_seq,tot_distance,company_id,vehicle_id,driver_id,participation_type,expnsn_factor,estab_taz,route_dur_min,has_warning,warn_BB_or_HH,warn_o_act_G,warn_o_act_S,warn_d_act_SG,warn_next_day,warn_duration,warn_no_BH_stops,warn_no_GS_stops,warn_no_GSM_stops
0,202209302062,Education/Other public services,LCV,4,0,1,1,2,0,Service,Non-Residential Only,2022-09-30 10:00:00,2022-09-30 14:20:00,NaN,B,B,Residential,Residential,"[B, M, B, S, B]","[1, 5, 1, 4, 1]",18.5,100012,2062,2589,online,177.825038,2284,260.0,0,0,0,0,0,0,0,0,0,0
1,202209302063,Education/Other public services,LCV,3,0,1,1,1,0,Service,Non-Residential Only,2022-09-30 14:00:00,2022-09-30 17:22:00,NaN,B,B,Residential,Residential,"[B, S, M, B]","[1, 4, 6, 1]",16.9,100012,2063,2590,online,177.825038,2284,202.0,0,0,0,0,0,0,0,0,0,0


In [123]:
out_file = dataset + f'_Routes_{current_date}.xlsx'
out_path = os.path.join(project_path, out_data_dir, out_file)
df_routes.to_excel(out_path, sheet_name='Routes', index=False) #  index_label='index'